# 모델링 전략

## 데이터 분할
    1. 2025년 5월 20일 데이터까지만 train, test로 사용하여 모델 평가 등 결과내기
    2. 2025년 5월 21일부터 31일까지 데이터를 지금은 정답을 가지고 있지만 실제는 없다고 가정하고 예측해보기(즉 합계_1일후, 등급_1일후를 실제 상황처럼 예측해보기)

## 모델링 방법
    1. 1일 후의 하수처리량을 각각 합계_1일후, 등급_1일후 컬럼을 생성
    2. 파생변수 만드는 함수 작성 -> 이건 추후에 새로운 데이터가 들어와도 알아서 계산될 수 잇도록 해야함
    3. 모델링은 분류, 회귀 각각에 대해 randomforest, xgboost, catboost, lightgbm, gradientboost 개씩 평가 (각각 결과를 테이블 형태로 정리하고 시각화 비교자료)
    4. 센터별 분류, 회귀 모델별로 가장 좋은 성능을 보이는 모델에 대해(좋은 모델에 대한 기준이 있어야함) 새로운데이터가 들어와도 예측할 수 있도록 해야함
    

In [56]:
import os, sys, platform, random, time, json, warnings
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import joblib

# sklearn
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    RandomForestClassifier, GradientBoostingClassifier,
)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, f1_score, classification_report, confusion_matrix,
)

# 선택적 라이브러리 확인
try:
    import xgboost as xgb; HAS_XGB = True
except ImportError:
    HAS_XGB = False; print("XGBoost 미설치 → 건너뜀")
try:
    import lightgbm as lgb; HAS_LGB = True
except ImportError:
    HAS_LGB = False; print("LightGBM 미설치 → 건너뜀")
try:
    import catboost as cb; HAS_CATBOOST = True
except ImportError:
    HAS_CATBOOST = False; print("CatBoost 미설치 → 건너뜀")
try:
    import shap; HAS_SHAP = True
except ImportError:
    HAS_SHAP = False; print("SHAP 미설치 → 건너뜀")

# 경고 필터
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")

try:
    plt.rcParams['font.family'] = 'AppleGothic' # 맥
except Exception:
    plt.rcParams['font.family'] ='Malgun Gothic' # 윈도우
plt.rcParams['axes.unicode_minus'] = False

In [57]:
# ================================================================================================
# 2. 모델 정의 함수들 (오류 수정됨)
# ================================================================================================
def build_regression_models():
    """회귀 모델들"""
    models = {}
    
    models["RandomForest_Reg"] = RandomForestRegressor(
        n_estimators=300, min_samples_leaf=2, random_state=42, n_jobs=-1
    )
    
    models["LinearRegression"] = LinearRegression()
    
    models["GradientBoosting_Reg"] = GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    
    if HAS_XGB:
        models["XGBoost_Reg"] = xgb.XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=0
        )
    
    if HAS_LGB:
        models["LightGBM_Reg"] = lgb.LGBMRegressor(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=-1
        )
    
    # 수정: 회귀 모델로 정정
    if HAS_CATBOOST:
        models["CatBoost_Reg"] = cb.CatBoostRegressor(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False
        )
    
    return models

def build_classification_models():
    """분류 모델들 (4등급)"""
    models = {}
    
    models["RandomForest_Clf"] = RandomForestClassifier(
        n_estimators=300, min_samples_leaf=2, random_state=42, 
        n_jobs=-1, class_weight='balanced'
    )
    
    models["GradientBoosting_Clf"] = GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    
    models["LogisticRegression_Clf"] = LogisticRegression(
        multi_class="multinomial", solver="lbfgs", max_iter=1000,
        random_state=42, class_weight='balanced'
    )
    
    if HAS_XGB:
        models["XGBoost_Clf"] = xgb.XGBClassifier(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multi:softprob", num_class=4,
            tree_method="hist", random_state=42, n_jobs=-1, verbosity=0
        )
    
    if HAS_LGB:
        models["LightGBM_Clf"] = lgb.LGBMClassifier(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multiclass", num_class=4,
            random_state=42, n_jobs=-1, verbosity=-1, is_unbalance=True
        )
    
    # 수정: auto_class_weights로 정정
    if HAS_CATBOOST:
        models["CatBoost_Clf"] = cb.CatBoostClassifier(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False, auto_class_weights='Balanced'
        )
    
    return models

# ================================================================================================
# 3. 결과 저장 시스템 (새로 추가)
# ================================================================================================
def create_result_directories():
    """결과 저장용 디렉토리 생성"""
    base_dir = '../data/results'
    subdirs = [
        'stratified_comparison',
        'feature_importance', 
        'shap_analysis',
        'model_performance',
        'visualizations'
    ]
    
    for subdir in [base_dir] + [os.path.join(base_dir, d) for d in subdirs]:
        os.makedirs(subdir, exist_ok=True)
    
    return base_dir

def save_results_comprehensive(results_df, analysis_type='stratified_comparison', 
                             center_name=None, model_name=None, extra_data=None):
    """포괄적 결과 저장 함수"""
    if len(results_df) == 0:
        print("저장할 결과가 없습니다.")
        return None
    
    # 디렉토리 생성
    base_dir = create_result_directories()
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    
    saved_files = []
    
    if analysis_type == 'stratified_comparison':
        # 1. 전체 실험 결과
        filename = f"stratified_comparison_{timestamp}.csv"
        filepath = os.path.join(base_dir, 'stratified_comparison', filename)
        results_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        saved_files.append(filepath)
        
        # 2. 성공한 결과만 필터링
        successful_results = results_df[results_df['success'] == True]
        
        if len(successful_results) > 0:
            # 3. 요약 통계
            summary_stats = create_summary_statistics(successful_results)
            if len(summary_stats) > 0:
                summary_filename = f"stratified_summary_{timestamp}.csv"
                summary_filepath = os.path.join(base_dir, 'stratified_comparison', summary_filename)
                summary_stats.to_csv(summary_filepath, index=False, encoding='utf-8-sig')
                saved_files.append(summary_filepath)
            
            # 4. 최고 성능 모델
            best_models = identify_best_models(successful_results)
            if len(best_models) > 0:
                best_filename = f"stratified_best_models_{timestamp}.csv"
                best_filepath = os.path.join(base_dir, 'stratified_comparison', best_filename)
                best_models.to_csv(best_filepath, index=False, encoding='utf-8-sig')
                saved_files.append(best_filepath)
    
    elif analysis_type == 'feature_importance':
        # 피처 중요도 저장
        filename = f"importance_{center_name}_{model_name}_{timestamp}.csv"
        filepath = os.path.join(base_dir, 'feature_importance', filename)
        results_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        saved_files.append(filepath)
    
    elif analysis_type == 'shap_analysis':
        # SHAP 값 저장 (pickle)
        if extra_data is not None and 'shap_values' in extra_data:
            shap_filename = f"shap_values_{center_name}_{model_name}_{timestamp}.pkl"
            shap_filepath = os.path.join(base_dir, 'shap_analysis', shap_filename)
            with open(shap_filepath, 'wb') as f:
                pickle.dump(extra_data['shap_values'], f)
            saved_files.append(shap_filepath)
        
        # SHAP 요약 정보 저장 (CSV)
        summary_filename = f"shap_summary_{center_name}_{model_name}_{timestamp}.csv"
        summary_filepath = os.path.join(base_dir, 'shap_analysis', summary_filename)
        results_df.to_csv(summary_filepath, index=False, encoding='utf-8-sig')
        saved_files.append(summary_filepath)
    
    elif analysis_type == 'model_performance':
        filename = f"performance_{timestamp}.csv"
        filepath = os.path.join(base_dir, 'model_performance', filename)
        results_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        saved_files.append(filepath)
    
    # 결과 출력
    print(f"\n=== 저장 완료 ===")
    for file in saved_files:
        print(f"저장됨: {file}")
    
    return saved_files

def create_summary_statistics(successful_results):
    """요약 통계 생성"""
    summary_stats = []
    
    for model_type in ['regression', 'classification']:
        type_data = successful_results[successful_results['type'] == model_type]
        if len(type_data) == 0:
            continue
            
        if model_type == 'regression':
            metrics = ['r2', 'mae', 'rmse', 'mape']
        else:
            metrics = ['accuracy', 'macro_f1', 'weighted_f1', 'extreme_f1']
        
        for metric in metrics:
            if metric in type_data.columns:
                # 모델별, 분할방법별 평균/표준편차
                grouped = type_data.groupby(['model', 'split_method'])[metric].agg(['mean', 'std', 'count']).round(4)
                grouped = grouped.reset_index()
                grouped['metric'] = metric
                grouped['type'] = model_type
                summary_stats.append(grouped)
    
    if summary_stats:
        return pd.concat(summary_stats, ignore_index=True)
    else:
        return pd.DataFrame()

def identify_best_models(successful_results):
    """센터별, 타입별 최고 성능 모델 식별"""
    best_models = []
    
    for center in successful_results['center'].unique():
        center_data = successful_results[successful_results['center'] == center]
        
        # 회귀 최고 성능 (R² 기준)
        reg_data = center_data[center_data['type'] == 'regression']
        if len(reg_data) > 0:
            best_reg_idx = reg_data['r2'].idxmax()
            best_reg = reg_data.loc[best_reg_idx].copy()
            best_reg['rank_type'] = 'regression_best'
            best_models.append(best_reg)
        
        # 분류 최고 성능 (Macro F1 기준)
        clf_data = center_data[center_data['type'] == 'classification']
        if len(clf_data) > 0:
            best_clf_idx = clf_data['macro_f1'].idxmax()
            best_clf = clf_data.loc[best_clf_idx].copy()
            best_clf['rank_type'] = 'classification_best'
            best_models.append(best_clf)
    
    if best_models:
        return pd.DataFrame(best_models)
    else:
        return pd.DataFrame()

def save_visualization(fig, filename):
    """시각화 결과 저장"""
    base_dir = create_result_directories()
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    
    filepath = os.path.join(base_dir, 'visualizations', f"{filename}_{timestamp}.png")
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"시각화 저장: {filepath}")
    return filepath

def create_performance_comparison_data(successful_results):
    """성능 비교 데이터 생성"""
    comparison_data = []
    
    # 센터별 성능 비교
    for center in successful_results['center'].unique():
        center_data = successful_results[successful_results['center'] == center]
        
        for split_method in center_data['split_method'].unique():
            method_data = center_data[center_data['split_method'] == split_method]
            
            # 회귀 평균 성능
            reg_data = method_data[method_data['type'] == 'regression']
            if len(reg_data) > 0:
                comparison_data.append({
                    'center': center,
                    'split_method': split_method,
                    'type': 'regression',
                    'avg_r2': reg_data['r2'].mean(),
                    'std_r2': reg_data['r2'].std(),
                    'avg_mae': reg_data['mae'].mean(),
                    'model_count': len(reg_data)
                })
            
            # 분류 평균 성능
            clf_data = method_data[method_data['type'] == 'classification']
            if len(clf_data) > 0:
                comparison_data.append({
                    'center': center,
                    'split_method': split_method,
                    'type': 'classification',
                    'avg_accuracy': clf_data['accuracy'].mean(),
                    'std_accuracy': clf_data['accuracy'].std(),
                    'avg_macro_f1': clf_data['macro_f1'].mean(),
                    'std_macro_f1': clf_data['macro_f1'].std(),
                    'avg_extreme_f1': clf_data['extreme_f1'].mean(),
                    'model_count': len(clf_data)
                })
    
    return pd.DataFrame(comparison_data)

# ================================================================================================
# 4. Feature Importance & SHAP 분석 함수들
# ================================================================================================
def extract_feature_importance(model, model_name, feature_names):
    """모델별 Feature Importance 추출"""
    try:
        mdl = model.named_steps['model']
        if hasattr(mdl, 'feature_importances_'):
            importance = mdl.feature_importances_
        elif hasattr(mdl, 'coef_'):
            coef = mdl.coef_
            # (n_features,) 또는 (n_classes, n_features) 모두 대응
            if isinstance(coef, np.ndarray) and coef.ndim == 2:
                importance = np.mean(np.abs(coef), axis=0)   # 클래스 평균
            else:
                importance = np.abs(coef)
        else:
            return None

        # 길이 안전가드
        if len(importance) != len(feature_names):
            print(f"[경고] importance 길이({len(importance)}) != feature_names({len(feature_names)}). "
                  "가능한 경우에만 앞쪽 피처로 맞춰서 반환합니다.")
            m = min(len(importance), len(feature_names))
            importance = np.asarray(importance)[:m]
            feature_names = list(feature_names)[:m]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return importance_df
    except Exception as e:
        print(f"Feature importance 추출 실패 ({model_name}): {e}")
        return None

def plot_feature_importance(importance_df, model_name, top_n=15):
    """Feature Importance 시각화 (fig 객체 리턴)"""
    # plot_feature_importance 자체가 새 figure를 생성하고 plt.show() 해버리기 때문에, fig는 사실상 빈 도화지예요. 그래서 저장하면 빈 그림이 찍히는 거죠.
    # 빈그림이 저장되어서 코드 수정한게 아래임
    if importance_df is None or len(importance_df) == 0:
        return None
    
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = importance_df.head(top_n)
    
    ax.barh(range(len(top_features)), top_features['importance'], color='skyblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{model_name} - Top {top_n} Feature Importance')
    ax.invert_yaxis()
    
    # 값 표시
    for i, v in enumerate(top_features['importance']):
        ax.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    fig.tight_layout()
    return fig   # ← 여기서 figure 객체 반환


def analyze_model_with_shap(model, X_test, feature_names, model_name, max_samples=100):
    """SHAP 분석"""
    if not HAS_SHAP:
        print("SHAP 라이브러리가 설치되지 않았습니다.")
        return None
    
    try:
        # 샘플 수 제한 (SHAP 계산 시간 단축)
        if len(X_test) > max_samples:
            sample_idx = np.random.choice(len(X_test), max_samples, replace=False)
            X_sample = X_test.iloc[sample_idx]
        else:
            X_sample = X_test
        
        # 전처리된 데이터 얻기
        X_processed = model.named_steps['pre'].transform(X_sample)
        
        # 모델별 SHAP Explainer 선택
        if 'RandomForest' in model_name or 'GradientBoosting' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'XGBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'LightGBM' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'CatBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        else:
            # Linear models
            explainer = shap.LinearExplainer(model.named_steps['model'], X_processed)
        
        # SHAP values 계산
        shap_values = explainer.shap_values(X_processed)
        
        # 다중 클래스의 경우 첫 번째 클래스만 사용
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        return shap_values, X_processed, explainer
        
    except Exception as e:
        print(f"SHAP 분석 실패 ({model_name}): {e}")
        return None

def plot_shap_summary(shap_values, X_processed, feature_names, model_name):
    """SHAP Summary Plot을 만들고 figure 목록을 반환 -> 빈그림 저장 방지"""
    if shap_values is None or not HAS_SHAP:
        return []

    figs = []
    try:
        # (1) Bar plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_processed,
                          feature_names=feature_names,
                          plot_type="bar", show=False)
        ax = plt.gca()
        ax.set_title(f'{model_name} - SHAP Feature Importance')
        figs.append(plt.gcf())

        # (2) Beeswarm plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_processed,
                          feature_names=feature_names,
                          show=False)
        ax = plt.gca()
        ax.set_title(f'{model_name} - SHAP Summary Plot')
        figs.append(plt.gcf())

    except Exception as e:
        print(f"SHAP 시각화 실패 ({model_name}): {e}")

    return figs

def comprehensive_model_analysis_with_save(model, model_name, X_train, X_test, y_train, y_test, 
                                         feature_names, model_type, center_name=None):
    """종합적인 모델 분석 (Feature Importance + SHAP) + 저장"""
    print(f"\n--- {model_name} 상세 분석 ---")
    
    # 1. Feature Importance 추출 및 시각화
    importance_df = extract_feature_importance(model, model_name, feature_names)
    if importance_df is not None:
        print(f"Top 10 중요 피처:")
        print(importance_df.head(10).to_string(index=False))
        
        # Feature Importance 저장
        if center_name:
            save_results_comprehensive(
                importance_df, 
                analysis_type='feature_importance',
                center_name=center_name,
                model_name=model_name
            )
        
        # 시각화
        fig = plot_feature_importance(importance_df, model_name)
        if fig is not None:
            fig.show()  # 화면에 띄우고
            if center_name:
                save_visualization(fig, f"feature_importance_{center_name}_{model_name}")
            plt.close(fig)

    
    # 2. SHAP 분석
    shap_result = analyze_model_with_shap(model, X_test, feature_names, model_name)
    if shap_result is not None:
        shap_values, X_processed, explainer = shap_result

        # SHAP 요약 통계 저장
        if isinstance(shap_values, np.ndarray):
            mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
            shap_summary_df = pd.DataFrame({
                'feature': feature_names,
                'mean_abs_shap': mean_abs_shap
            }).sort_values('mean_abs_shap', ascending=False)
            if center_name:
                extra_data = {'shap_values': shap_values, 'X_processed': X_processed}
                save_results_comprehensive(
                    shap_summary_df,
                    analysis_type='shap_analysis',
                    center_name=center_name,
                    model_name=model_name,
                    extra_data=extra_data
                )

        # SHAP 플롯 생성(리턴받음)
        shap_figs = plot_shap_summary(shap_values, X_processed, feature_names, model_name)
        if shap_figs:
            for i, fig in enumerate(shap_figs):
                fig.show()  # 또는 plt.show()
                if center_name:
                    suffix = "shap_bar" if i == 0 else "shap_beeswarm"
                    save_visualization(fig, f"{suffix}_{center_name}_{model_name}")
                plt.close(fig)  # 메모리 정리

    
    return importance_df, shap_result

# ================================================================================================
# 5. 데이터 처리 및 평가 함수들
# ================================================================================================
def make_pipeline_unified(model, model_name, model_type):
    """통합 전처리 파이프라인"""
    if model_name in ["LinearRegression", "LogisticRegression_Clf"]:
        # 선형 모델은 정규화 필요
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ])
    else:
        # 트리 기반 모델들은 정규화 불필요
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ])
    return Pipeline(steps=[("pre", pre), ("model", model)])

def prepare_data_stratified(df, target_col, model_type, test_size=0.2, split_method='stratified'):
    """
    데이터 준비 - Stratified vs 시계열 분할 선택 가능
    
    Parameters:
    - split_method: 'stratified' 또는 'temporal'
    """
    work = df.sort_values('날짜').reset_index(drop=True).copy()
    dates = pd.to_datetime(work['날짜'])

    # 제외할 컬럼들
    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in work.columns]
    X_raw = work.drop(columns=drop_cols, errors="ignore")
    
    # 수치형 변환
    for c in X_raw.columns:
        X_raw[c] = pd.to_numeric(X_raw[c], errors="coerce")

    if model_type == "regression":
        y = pd.to_numeric(work[target_col], errors="coerce")
    else:  # classification
        y = work[target_col].astype("int64")

    # 결측치 제거
    valid_idx = (~X_raw.isnull().all(axis=1)) & (~pd.isnull(y))
    X_raw = X_raw[valid_idx].reset_index(drop=True)
    y = y[valid_idx].reset_index(drop=True)
    dates = dates[valid_idx].reset_index(drop=True)
    
    if split_method == 'stratified':
        # Stratified 분할 (분류에만 적용, 회귀는 일반 random split)
        if model_type == "classification":
            # 등급별 균등 분할
            sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
            train_idx, test_idx = next(sss.split(X_raw, y))
        else:
            # 회귀는 일반 랜덤 분할 (연속값이므로 stratify 불가)
            train_idx, test_idx = train_test_split(
                range(len(X_raw)), test_size=test_size, random_state=42
            )
            
        X_train, X_test = X_raw.iloc[train_idx].copy(), X_raw.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        dates_train, dates_test = dates.iloc[train_idx].copy(), dates.iloc[test_idx].copy()
        
    else:  # temporal split
        # 기존 시계열 분할
        n = len(X_raw) # len(work)
        split = int(n * (1 - test_size))
        X_train, X_test = X_raw.iloc[:split].copy(), X_raw.iloc[split:].copy()
        y_train, y_test = y.iloc[:split].copy(), y.iloc[split:].copy()
        dates_train, dates_test = dates.iloc[:split].copy(), dates.iloc[split:].copy()

    feature_names = list(X_raw.columns)
    return X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test

def evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test):
    """회귀 모델 평가 (수정됨)"""
    try:
        pipe = make_pipeline_unified(model, model_name, "regression")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        # MAPE (Mean Absolute Percentage Error)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        # 올바른 return 문
        return {
            'model': model_name,
            'type': 'regression',
            'mae': mae,         # ← 계산된 값 사용
            'rmse': rmse,       # ← 계산된 값 사용
            'r2': r2,           # ← 계산된 값 사용
            'mape': mape,       # ← 계산된 값 사용
            'success': True     # ← 성공 시 True
        }, pipe, y_pred
        
    except Exception as e:
        # 실패 시에만 이 부분 실행
        return {
            'model': model_name,
            'type': 'regression',
            'mae': np.nan,
            'rmse': np.nan,
            'r2': np.nan,
            'mape': np.nan,
            'success': False,
            'error': str(e)
        }, None, None
        
def evaluate_classification_model(model, model_name, X_train, X_test, y_train, y_test):
    """분류 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "classification")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        # 차원 문제 해결
        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
            y_pred = y_pred.ravel()
        
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)
        f1_weighted = f1_score(y_test, y_pred, average="weighted", zero_division=0)
        
        # 극값 분류 성능 (등급 0, 3로 수정: 0-base 변환 때문에)
        extreme_classes = [0, 3]  # 원래 1,4가 0,3으로 변환됨
        y_true_extreme = pd.Series(y_test).isin(extreme_classes).astype(int)
        y_pred_extreme = pd.Series(y_pred).isin(extreme_classes).astype(int)
        extreme_f1 = f1_score(y_true_extreme, y_pred_extreme, zero_division=0)
        
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': acc,
            'macro_f1': f1_macro,
            'weighted_f1': f1_weighted,
            'extreme_f1': extreme_f1,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': np.nan,
            'macro_f1': np.nan,
            'weighted_f1': np.nan,
            'extreme_f1': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def comprehensive_evaluation_comparison(center_name, df):
    """Stratified vs 시계열 분할 비교 평가"""
    print(f"\n{'='*70}")
    print(f"센터: {center_name} - Stratified vs 시계열 분할 비교")
    print(f"{'='*70}")
    
    # 데이터 확인
    print(f"데이터 크기: {len(df)}행, {len(df.columns)}컬럼")
    
    # 등급 분포 확인
    if '등급_1일후' in df.columns:
        grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"등급 분포: {dict(grade_dist)}")
        
        # 불균형 정도 확인
        min_class = grade_dist.min()
        max_class = grade_dist.max()
        imbalance_ratio = max_class / min_class
        print(f"클래스 불균형 비율: {imbalance_ratio:.1f}:1 (최대:{max_class}, 최소:{min_class})")
    
    results = []
    
    # 두 가지 분할 방법 비교
    for split_method in ['temporal', 'stratified']:
        print(f"\n{'='*50}")
        print(f"분할 방법: {split_method.upper()}")
        print(f"{'='*50}")
        
        # =========================
        # 1. 회귀 모델 평가
        # =========================
        reg_method_name = "random_shuffle" if split_method == "stratified" else split_method
        print(f"\n--- 회귀 모델 평가 ({reg_method_name}) ---")
        
        try:
            X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test = prepare_data_stratified(
                df, target_col="합계_1일후", model_type="regression", test_size=0.2, split_method=split_method
            )
            
            print(f"회귀용 데이터: 학습 {len(X_train)}행, 테스트 {len(X_test)}행")
            
            regression_models = build_regression_models()
            
            for model_name, model in tqdm(regression_models.items(), desc=f"회귀({reg_method_name})", leave=False):
                result, pipe, y_pred = evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: R²={result['r2']:.3f}, MAE={result['mae']:.0f}, MAPE={result['mape']:.1f}%")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"회귀 모델 평가 실패 ({reg_method_name}): {e}")
        
        # =========================
        # 2. 분류 모델 평가
        # =========================
        print(f"\n--- 분류 모델 평가 ({split_method}) ---")
        
        try:
            X_train_clf, X_test_clf, y_train_clf, y_test_clf, feature_names_clf, _, _ = prepare_data_stratified(
                df, target_col="등급_1일후", model_type="classification", test_size=0.2, split_method=split_method
            )
            
            print(f"분류용 데이터: 학습 {len(X_train_clf)}행, 테스트 {len(X_test_clf)}행")
            
            # 테스트 세트 등급 분포 확인
            test_dist = pd.Series(y_test_clf).value_counts().sort_index()
            train_dist = pd.Series(y_train_clf).value_counts().sort_index()
            print(f"학습 세트 등급 분포: {dict(train_dist)}")
            print(f"테스트 세트 등급 분포: {dict(test_dist)}")
            
            classification_models = build_classification_models()
            
            for model_name, model in tqdm(classification_models.items(), desc=f"분류({split_method})", leave=False):
                result, pipe, y_pred = evaluate_classification_model(model, model_name, X_train_clf, X_test_clf, y_train_clf, y_test_clf)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: ACC={result['accuracy']:.3f}, F1={result['macro_f1']:.3f}, 극값F1={result['extreme_f1']:.3f}")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"분류 모델 평가 실패 ({split_method}): {e}")
    
    return results

# ================================================================================================
# 6. 시각화 함수들
# ================================================================================================
def plot_stratified_comparison(results_df):
    """비교 결과 시각화"""
    if len(results_df) == 0:
        print("시각화할 결과가 없습니다.")
        return
    
    # successful_results가 results_df 안에 있어야 함
    successful_results = results_df[results_df['success'] == True].copy()
    if len(successful_results) == 0:
        print("성공한 결과가 없어 시각화를 생략합니다.")
        return

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Stratified vs 시계열 분할 비교', fontsize=16)
    
    # 1. 회귀 모델 R² 비교
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        reg_pivot = reg_results.pivot_table(
            index='model', columns='split_method', values='r2', aggfunc='mean'
        )
        reg_pivot = reg_pivot.rename(columns={'stratified': 'random_shuffle'})
        reg_pivot.plot(kind='bar', ax=axes[0,0], color=['lightblue', 'orange'])
        axes[0,0].set_title('회귀 모델 R² 성능 비교')
        axes[0,0].set_ylabel('R² Score')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].legend(title='Split Method')
        axes[0,0].set_ylim(0, 1)
    
    # 2. 분류 모델 F1 비교
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        clf_pivot = clf_results.pivot_table(
            index='model', columns='split_method', values='macro_f1', aggfunc='mean'
        )
        clf_pivot.plot(kind='bar', ax=axes[0,1], color=['lightgreen', 'red'])
        axes[0,1].set_title('분류 모델 Macro F1 성능 비교')
        axes[0,1].set_ylabel('Macro F1 Score')
        axes[0,1].tick_params(axis='x', rotation=45)
        axes[0,1].legend(title='Split Method')
        axes[0,1].set_ylim(0, 1)
    
    # 3. 센터별 회귀 성능
    if len(reg_results) > 0:
        center_reg = reg_results.groupby(['center', 'split_method'])['r2'].mean().unstack()
        center_reg = center_reg.rename(columns={'stratified': 'random_shuffle'})
        center_reg.plot(kind='bar', ax=axes[1,0], color=['lightblue', 'orange'])
        axes[1,0].set_title('센터별 회귀 평균 성능')
        axes[1,0].set_ylabel('평균 R² Score')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].legend(title='Split Method')
    
    # 4. 센터별 분류 성능
    if len(clf_results) > 0:
        center_clf = clf_results.groupby(['center', 'split_method'])['macro_f1'].mean().unstack()
        center_clf.plot(kind='bar', ax=axes[1,1], color=['lightgreen', 'red'])
        axes[1,1].set_title('센터별 분류 평균 성능')
        axes[1,1].set_ylabel('평균 Macro F1 Score')
        axes[1,1].tick_params(axis='x', rotation=45)
        axes[1,1].legend(title='Split Method')
    
    plt.tight_layout()
    plt.show()
    
    # 시각화 저장
    save_visualization(fig, "stratified_comparison_plots")

def plot_model_performance_comparison(results_df):
    """모델별 성능 상세 비교 시각화"""
    if len(results_df) == 0:
        return
        
    successful_results = results_df[results_df['success'] == True].copy()
    
    # 1. 모든 모델 성능 한눈에 보기
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('모델별 성능 상세 비교', fontsize=16)
    
    # 1-1. 회귀 모델 R² (분할 방법별)
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        reg_pivot = reg_results.pivot_table(
            index='model', columns='split_method', values='r2', aggfunc='mean'
        )
        reg_pivot.plot(kind='bar', ax=axes[0,0], color=['lightblue', 'orange'])
        axes[0,0].set_title('회귀 모델 R² 성능')
        axes[0,0].set_ylabel('R² Score')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].legend(title='Split Method')
    
    # 1-2. 분류 모델 Macro F1 (분할 방법별)
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        clf_pivot = clf_results.pivot_table(
            index='model', columns='split_method', values='macro_f1', aggfunc='mean'
        )
        clf_pivot.plot(kind='bar', ax=axes[0,1], color=['lightgreen', 'red'])
        axes[0,1].set_title('분류 모델 Macro F1 성능')
        axes[0,1].set_ylabel('Macro F1 Score')
        axes[0,1].tick_params(axis='x', rotation=45)
        axes[0,1].legend(title='Split Method')
    
    # 1-3. 분류 모델 Extreme F1 성능
    if len(clf_results) > 0:
        extreme_pivot = clf_results.pivot_table(
            index='model', columns='split_method', values='extreme_f1', aggfunc='mean'
        )
        extreme_pivot.plot(kind='bar', ax=axes[0,2], color=['lightcoral', 'gold'])
        axes[0,2].set_title('분류 모델 Extreme F1 성능')
        axes[0,2].set_ylabel('Extreme F1 Score')
        axes[0,2].tick_params(axis='x', rotation=45)
        axes[0,2].legend(title='Split Method')
    
    # 2-1. 회귀 모델 MAE 성능
    if len(reg_results) > 0:
        mae_pivot = reg_results.pivot_table(
            index='model', columns='split_method', values='mae', aggfunc='mean'
        )
        mae_pivot.plot(kind='bar', ax=axes[1,0], color=['lightsteelblue', 'sandybrown'])
        axes[1,0].set_title('회귀 모델 MAE 성능 (낮을수록 좋음)')
        axes[1,0].set_ylabel('Mean Absolute Error')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].legend(title='Split Method')
    
    # 2-2. 분류 모델 Accuracy
    if len(clf_results) > 0:
        acc_pivot = clf_results.pivot_table(
            index='model', columns='split_method', values='accuracy', aggfunc='mean'
        )
        acc_pivot.plot(kind='bar', ax=axes[1,1], color=['mediumseagreen', 'indianred'])
        axes[1,1].set_title('분류 모델 Accuracy 성능')
        axes[1,1].set_ylabel('Accuracy Score')
        axes[1,1].tick_params(axis='x', rotation=45)
        axes[1,1].legend(title='Split Method')
    
    # 2-3. 센터별 최고 성능 모델
    plot_best_models_per_center(successful_results, axes[1,2])
    
    plt.tight_layout()
    plt.show()
    
    # 시각화 저장
    save_visualization(fig, "model_performance_comparison")

def plot_best_models_per_center(results_df, ax):
    """센터별 최고 성능 모델 표시"""
    centers = results_df['center'].unique()
    reg_best = []
    clf_best = []
    
    for center in centers:
        center_data = results_df[results_df['center'] == center]
        
        # 회귀 최고 성능
        reg_data = center_data[center_data['type'] == 'regression']
        if len(reg_data) > 0:
            best_reg_idx = reg_data['r2'].idxmax()
            reg_best.append(reg_data.loc[best_reg_idx, 'r2'])
        else:
            reg_best.append(0)
        
        # 분류 최고 성능
        clf_data = center_data[center_data['type'] == 'classification']
        if len(clf_data) > 0:
            best_clf_idx = clf_data['macro_f1'].idxmax()
            clf_best.append(clf_data.loc[best_clf_idx, 'macro_f1'])
        else:
            clf_best.append(0)
    
    x = np.arange(len(centers))
    width = 0.35
    
    ax.bar(x - width/2, reg_best, width, label='회귀 R²', color='skyblue')
    ax.bar(x + width/2, clf_best, width, label='분류 F1', color='lightcoral')
    
    ax.set_xlabel('센터')
    ax.set_ylabel('성능 점수')
    ax.set_title('센터별 최고 성능')
    ax.set_xticks(x)
    ax.set_xticklabels(centers)
    ax.legend()
    ax.set_ylim(0, 1)
    
    # 값 표시
    for i, (r, c) in enumerate(zip(reg_best, clf_best)):
        if r > 0:
            ax.text(i - width/2, r + 0.01, f'{r:.3f}', ha='center', va='bottom')
        if c > 0:
            ax.text(i + width/2, c + 0.01, f'{c:.3f}', ha='center', va='bottom')

# ================================================================================================
# 7. 분석 함수들
# ================================================================================================
def analyze_stratified_comparison(results_df):
    """Stratified vs 시계열 비교 분석"""
    print(f"\n{'='*60}")
    print("=== Stratified vs 시계열 분할 비교 분석 ===")
    print(f"{'='*60}")
    
    successful_results = results_df[results_df['success'] == True].copy()
    
    # 분할 방법별 성능 비교
    for task_type in ['regression', 'classification']:
        print(f"\n--- {task_type.upper()} 모델 비교 ---")
        
        task_results = successful_results[successful_results['type'] == task_type]
        if len(task_results) == 0:
            continue
            
        if task_type == 'regression':
            metric = 'r2'
            metric_name = 'R²'
        else:
            metric = 'macro_f1'  
            metric_name = 'Macro F1'
        
        # 분할 방법별 평균 성능
        split_performance = task_results.groupby('split_method')[metric].agg(['mean', 'std', 'count'])
        
        for split_method in split_performance.index:
            mean_val = split_performance.loc[split_method, 'mean']
            std_val = split_performance.loc[split_method, 'std']
            count_val = split_performance.loc[split_method, 'count']
            method_display = "random_shuffle" if split_method == "stratified" and task_type == "regression" else split_method
            print(f"  {method_display:13s}: {metric_name}={mean_val:.3f} ± {std_val:.3f} ({count_val}개)")
        
        # 모델별 비교
        print(f"\n  모델별 {metric_name} 비교:")
        model_comparison = task_results.pivot_table(
            index='model', columns='split_method', values=metric, aggfunc='mean'
        ).round(3)
        
        print(model_comparison.to_string())
        
        # 개선 정도 분석 (stratified가 temporal보다 좋은 경우)
        if 'temporal' in model_comparison.columns and 'stratified' in model_comparison.columns:
            improvement = model_comparison['stratified'] - model_comparison['temporal']
            improvement_name = "Random Shuffle 개선 정도" if task_type == "regression" else "Stratified 개선 정도"
            print(f"\n  {improvement_name} ({metric_name}):")
            for model in improvement.index:
                imp_val = improvement[model]
                if not pd.isna(imp_val):
                    symbol = "↑" if imp_val > 0 else "↓" if imp_val < 0 else "="
                    print(f"    {model:18s}: {imp_val:+.3f} {symbol}")
    
    # 센터별 분할 방법 효과
    print(f"\n--- 센터별 분할 방법 효과 ---")
    for center in successful_results['center'].unique():
        center_data = successful_results[successful_results['center'] == center]
        
        reg_data = center_data[center_data['type'] == 'regression']
        clf_data = center_data[center_data['type'] == 'classification']
        
        print(f"\n  {center.upper()} 센터:")
        
        # 회귀 성능
        if len(reg_data) > 0:
            reg_perf = reg_data.groupby('split_method')['r2'].mean()
            for method in reg_perf.index:
                method_display = "random_shuffle" if method == "stratified" else method
                print(f"    회귀 R² ({method_display:13s}): {reg_perf[method]:.3f}")
        
        # 분류 성능  
        if len(clf_data) > 0:
            clf_perf = clf_data.groupby('split_method')['macro_f1'].mean()
            for method in clf_perf.index:
                print(f"    분류 F1 ({method:13s}): {clf_perf[method]:.3f}")
    
    # 기본 시각화
    plot_stratified_comparison(successful_results)
    
    # 상세 모델 성능 비교
    plot_model_performance_comparison(successful_results)

def perform_detailed_analysis_with_save(results_df, centers):
    """최고 성능 모델에 대한 상세 분석 (저장 기능 포함)"""
    print(f"\n{'='*60}")
    print("=== 최고 성능 모델 상세 분석 ===")
    print(f"{'='*60}")
    
    successful_results = results_df[results_df['success'] == True].copy()
    
    # 최고 성능 모델 찾기
    reg_results = successful_results[successful_results['type'] == 'regression']
    clf_results = successful_results[successful_results['type'] == 'classification']
    
    analyzed_models = []
    
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"\n최고 회귀 성능: {best_reg['center']} - {best_reg['model']} ({best_reg['split_method']}) R²={best_reg['r2']:.3f}")
        
        # 최고 성능 모델 재학습 및 분석
        model_info = analyze_best_model_with_save(best_reg, centers, 'regression')
        if model_info:
            analyzed_models.append(model_info)
    
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"\n최고 분류 성능: {best_clf['center']} - {best_clf['model']} ({best_clf['split_method']}) F1={best_clf['macro_f1']:.3f}")
        
        # 최고 성능 모델 재학습 및 분석
        model_info = analyze_best_model_with_save(best_clf, centers, 'classification')
        if model_info:
            analyzed_models.append(model_info)
    
    return analyzed_models

def analyze_best_model_with_save(best_result, centers, model_type):
    """최고 성능 모델 상세 분석 (저장 기능 포함)"""
    center_name = best_result['center']
    model_name = best_result['model']
    split_method = best_result['split_method']
    
    print(f"\n{'='*50}")
    print(f"모델 재학습 및 분석: {center_name} - {model_name}")
    print(f"{'='*50}")
    
    try:
        # 데이터 준비
        df = centers[center_name]
        target_col = "합계_1일후" if model_type == "regression" else "등급_1일후"
        
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            df, target_col=target_col, model_type=model_type, 
            test_size=0.2, split_method=split_method
        )
        
        # 모델 재구축 및 학습
        if model_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
            
        model = models[model_name]
        pipe = make_pipeline_unified(model, model_name, model_type)
        pipe.fit(X_train, y_train)
        
        # 상세 분석 수행 (저장 포함)
        importance_df, shap_result = comprehensive_model_analysis_with_save(
            pipe, model_name, X_train, X_test, y_train, y_test, 
            feature_names, model_type, center_name
        )
        
        return {
            'center': center_name,
            'model': model_name,
            'type': model_type,
            'split_method': split_method,
            'performance': best_result,
            'analysis_completed': True
        }
        
    except Exception as e:
        print(f"상세 분석 실패: {e}")
        return None

# ================================================================================================
# 8. 메인 실행 함수들 (개선된 버전)
# ================================================================================================
def run_stratified_comparison():
    """전체 센터 Stratified vs 시계열 비교 (개선된 저장 기능)"""
    print("=== Stratified vs 시계열 분할 비교 실험 ===")
    
    # 데이터 로드 확인
    try:
        centers = {
            "nanji": nanji,
            "jungnang": jungnang,  
            "seonam": seonam,
            "tancheon": tancheon
        }
        
        print(f"\n데이터 확인:")
        for name, df in centers.items():
            print(f"  {name}: {len(df)}행")
    
    except NameError:
        print("데이터가 로드되지 않았습니다. 먼저 make_features를 실행하세요.")
        return pd.DataFrame()
    
    # 전체 실험 실행
    all_results = []
    
    for center_name, df in centers.items():
        try:
            center_results = comprehensive_evaluation_comparison(center_name, df)
            all_results.extend(center_results)
        except Exception as e:
            print(f"[{center_name}] 실험 실패: {e}")
    
    # 결과 분석
    results_df = pd.DataFrame(all_results)
    
    if len(results_df) > 0:
        # 분석 및 시각화
        analyze_stratified_comparison(results_df)
        
        # 최고 성능 모델 식별 및 상세 분석
        perform_detailed_analysis_with_save(results_df, centers)
        
        # 포괄적 결과 저장
        saved_files = save_results_comprehensive(results_df, 'stratified_comparison')
        
        # 모델 성능 비교 데이터도 별도 저장
        successful_results = results_df[results_df['success'] == True]
        if len(successful_results) > 0:
            performance_comparison = create_performance_comparison_data(successful_results)
            save_results_comprehensive(
                performance_comparison, 
                'model_performance'
            )
        
        print(f"\n=== 실험 완료 ===")
        print(f"총 {len(results_df)}개 실험 중 {len(successful_results)}개 성공")
        print(f"결과 저장 위치: ../data/results/")
    
    return results_df

def quick_stratified_test(center_name="nanji"):
    """단일 센터 Stratified 테스트"""
    print(f"=== {center_name} 센터 Stratified vs 시계열 비교 ===")
    
    try:
        if center_name == "nanji":
            df = nanji
        elif center_name == "jungnang":
            df = jungnang
        elif center_name == "seonam":
            df = seonam
        elif center_name == "tancheon":
            df = tancheon
        else:
            raise ValueError(f"Unknown center: {center_name}")
            
        results = comprehensive_evaluation_comparison(center_name, df)
        return pd.DataFrame(results)
        
    except Exception as e:
        print(f"테스트 실패: {e}")
        return pd.DataFrame()
    
# ================================================================================================
# 모델 저장 함수들 (기존 코드에 추가)
# ================================================================================================

def save_trained_model(model_pipeline, model_info, performance_metrics, feature_names, 
                      center_name, model_name, split_method):
    """학습된 모델과 모든 관련 정보를 저장"""
    
    # 저장 디렉토리 생성
    base_dir = create_result_directories()
    model_dir = os.path.join(base_dir, 'trained_models')
    os.makedirs(model_dir, exist_ok=True)
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    
    # 파일명 생성
    model_filename = f"{center_name}_{model_name}_{split_method}_{timestamp}"
    
    saved_files = []
    
    try:
        # 1. 모델 파이프라인 저장 (joblib - sklearn 모델에 최적화)
        model_path = os.path.join(model_dir, f"{model_filename}_model.pkl")
        joblib.dump(model_pipeline, model_path)
        saved_files.append(model_path)
        
        # 2. 모델 메타데이터 저장 (JSON)
        metadata = {
            'model_info': {
                'center_name': center_name,
                'model_name': model_name,
                'model_type': model_info.get('type', 'unknown'),
                'split_method': split_method,
                'training_timestamp': timestamp,
                'training_date': datetime.now().isoformat()
            },
            'data_info': {
                'feature_names': feature_names,
                'feature_count': len(feature_names),
                'target_column': "합계_1일후" if model_info.get('type') == 'regression' else "등급_1일후"
            },
            'preprocessing_info': {
                'imputer_strategy': 'median',
                'scaling_applied': model_name in ["LinearRegression", "LogisticRegression_Clf"],
                'pipeline_steps': ['imputer'] + (['scaler'] if model_name in ["LinearRegression", "LogisticRegression_Clf"] else [])
            },
            'performance_metrics': performance_metrics,
            'model_parameters': get_model_parameters(model_pipeline, model_name)
        }
        
        metadata_path = os.path.join(model_dir, f"{model_filename}_metadata.json")
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        saved_files.append(metadata_path)
        
        # 3. 피처 이름 리스트 별도 저장 (빠른 접근용)
        feature_path = os.path.join(model_dir, f"{model_filename}_features.txt")
        with open(feature_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(feature_names))
        saved_files.append(feature_path)
        
        print(f"\n=== 모델 저장 완료 ===")
        print(f"모델: {center_name} - {model_name} ({split_method})")
        for file in saved_files:
            print(f"저장됨: {file}")
            
        return model_filename, saved_files
        
    except Exception as e:
        print(f"모델 저장 실패: {e}")
        return None, []

def get_model_parameters(model_pipeline, model_name):
    """모델의 하이퍼파라미터 추출"""
    try:
        model = model_pipeline.named_steps['model']
        params = model.get_params()
        
        # 중요한 파라미터만 저장 (너무 많으면 파일이 커짐)
        important_params = {}
        
        if 'RandomForest' in model_name:
            important_params = {
                'n_estimators': params.get('n_estimators'),
                'max_depth': params.get('max_depth'), 
                'min_samples_leaf': params.get('min_samples_leaf'),
                'random_state': params.get('random_state')
            }
        elif 'XGBoost' in model_name:
            important_params = {
                'n_estimators': params.get('n_estimators'),
                'max_depth': params.get('max_depth'),
                'learning_rate': params.get('learning_rate'),
                'subsample': params.get('subsample'),
                'random_state': params.get('random_state')
            }
        elif 'LightGBM' in model_name:
            important_params = {
                'n_estimators': params.get('n_estimators'),
                'learning_rate': params.get('learning_rate'),
                'random_state': params.get('random_state')
            }
        elif 'CatBoost' in model_name:
            important_params = {
                'iterations': params.get('iterations'),
                'learning_rate': params.get('learning_rate'),
                'depth': params.get('depth'),
                'random_state': params.get('random_state')
            }
        else:
            # Linear models 등
            important_params = {key: val for key, val in params.items() 
                              if key in ['C', 'max_iter', 'random_state', 'solver']}
        
        return important_params
        
    except Exception as e:
        print(f"파라미터 추출 실패: {e}")
        return {}

def load_trained_model(model_filename, model_dir=None):
    """저장된 모델과 메타데이터 로드"""
    
    if model_dir is None:
        base_dir = create_result_directories()
        model_dir = os.path.join(base_dir, 'trained_models')
    
    try:
        # 1. 모델 로드
        model_path = os.path.join(model_dir, f"{model_filename}_model.pkl")
        model_pipeline = joblib.load(model_path)
        
        # 2. 메타데이터 로드
        metadata_path = os.path.join(model_dir, f"{model_filename}_metadata.json")
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        
        # 3. 피처 이름 로드
        feature_path = os.path.join(model_dir, f"{model_filename}_features.txt")
        with open(feature_path, 'r', encoding='utf-8') as f:
            feature_names = [line.strip() for line in f.readlines()]
        
        print(f"모델 로드 완료: {metadata['model_info']['center_name']} - {metadata['model_info']['model_name']}")
        print(f"학습 일시: {metadata['model_info']['training_date']}")
        print(f"성능 지표: {metadata['performance_metrics']}")
        
        return {
            'model_pipeline': model_pipeline,
            'metadata': metadata,
            'feature_names': feature_names
        }
        
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        return None

def predict_with_saved_model(model_data, new_data):
    """저장된 모델로 새로운 데이터 예측"""
    
    try:
        model_pipeline = model_data['model_pipeline']
        expected_features = model_data['feature_names']
        metadata = model_data['metadata']
        
        # 1. 피처 확인 및 정렬
        if isinstance(new_data, pd.DataFrame):
            # 필요한 피처만 선택하고 순서 맞춤
            missing_features = set(expected_features) - set(new_data.columns)
            if missing_features:
                print(f"경고: 다음 피처들이 누락됨: {missing_features}")
                # 누락된 피처는 0으로 채움
                for feature in missing_features:
                    new_data[feature] = 0
            
            # 피처 순서 맞춤
            X_new = new_data[expected_features].copy()
        else:
            raise ValueError("새 데이터는 pandas DataFrame이어야 합니다.")
        
        # 2. 예측 수행
        predictions = model_pipeline.predict(X_new)
        
        # 3. 예측 결과 후처리
        model_type = metadata['model_info']['model_type']
        
        if model_type == 'classification':
            # 분류의 경우 확률도 함께 반환
            try:
                probabilities = model_pipeline.predict_proba(X_new)
                return {
                    'predictions': predictions,
                    'probabilities': probabilities,
                    'model_type': model_type,
                    'model_name': metadata['model_info']['model_name']
                }
            except:
                return {
                    'predictions': predictions,
                    'model_type': model_type,
                    'model_name': metadata['model_info']['model_name']
                }
        else:
            # 회귀의 경우
            return {
                'predictions': predictions,
                'model_type': model_type,
                'model_name': metadata['model_info']['model_name']
            }
            
    except Exception as e:
        print(f"예측 실패: {e}")
        return None

def list_saved_models(model_dir=None):
    """저장된 모델 목록 조회"""
    
    if model_dir is None:
        base_dir = create_result_directories()
        model_dir = os.path.join(base_dir, 'trained_models')
    
    if not os.path.exists(model_dir):
        print("저장된 모델이 없습니다.")
        return []
    
    model_files = [f for f in os.listdir(model_dir) if f.endswith('_metadata.json')]
    models_info = []
    
    for metadata_file in model_files:
        try:
            metadata_path = os.path.join(model_dir, metadata_file)
            with open(metadata_path, 'r', encoding='utf-8') as f:
                metadata = json.load(f)
            
            model_filename = metadata_file.replace('_metadata.json', '')
            
            info = {
                'filename': model_filename,
                'center': metadata['model_info']['center_name'],
                'model': metadata['model_info']['model_name'],
                'type': metadata['model_info']['model_type'],
                'split_method': metadata['model_info']['split_method'],
                'training_date': metadata['model_info']['training_date'],
                'performance': metadata['performance_metrics']
            }
            models_info.append(info)
            
        except Exception as e:
            print(f"메타데이터 읽기 실패 ({metadata_file}): {e}")
    
    return models_info

# ================================================================================================
# 기존 평가 함수들 수정 (모델 저장 기능 추가)
# ================================================================================================

def evaluate_regression_model_with_save(model, model_name, X_train, X_test, y_train, y_test, 
                                       feature_names=None, center_name=None, split_method=None, 
                                       save_model=False):
    """회귀 모델 평가 + 모델 저장 옵션"""
    try:
        pipe = make_pipeline_unified(model, model_name, "regression")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        performance_metrics = {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape
        }
        
        result = {
            'model': model_name,
            'type': 'regression',
            **performance_metrics,
            'success': True
        }
        
        # 모델 저장 (요청 시)
        saved_model_info = None
        if save_model and feature_names and center_name and split_method:
            model_info = {'type': 'regression'}
            filename, files = save_trained_model(
                pipe, model_info, performance_metrics, feature_names,
                center_name, model_name, split_method
            )
            saved_model_info = {'filename': filename, 'files': files}
        
        return result, pipe, y_pred, saved_model_info
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'regression',
            'mae': np.nan,
            'rmse': np.nan,
            'r2': np.nan,
            'mape': np.nan,
            'success': False,
            'error': str(e)
        }, None, None, None

def evaluate_classification_model_with_save(model, model_name, X_train, X_test, y_train, y_test,
                                          feature_names=None, center_name=None, split_method=None,
                                          save_model=False):
    """분류 모델 평가 + 모델 저장 옵션"""
    try:
        pipe = make_pipeline_unified(model, model_name, "classification")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
            y_pred = y_pred.ravel()
        
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)
        f1_weighted = f1_score(y_test, y_pred, average="weighted", zero_division=0)
        
        extreme_classes = [0, 3]
        y_true_extreme = pd.Series(y_test).isin(extreme_classes).astype(int)
        y_pred_extreme = pd.Series(y_pred).isin(extreme_classes).astype(int)
        extreme_f1 = f1_score(y_true_extreme, y_pred_extreme, zero_division=0)
        
        performance_metrics = {
            'accuracy': acc,
            'macro_f1': f1_macro,
            'weighted_f1': f1_weighted,
            'extreme_f1': extreme_f1
        }
        
        result = {
            'model': model_name,
            'type': 'classification',
            **performance_metrics,
            'success': True
        }
        
        # 모델 저장 (요청 시)
        saved_model_info = None
        if save_model and feature_names and center_name and split_method:
            model_info = {'type': 'classification'}
            filename, files = save_trained_model(
                pipe, model_info, performance_metrics, feature_names,
                center_name, model_name, split_method
            )
            saved_model_info = {'filename': filename, 'files': files}
        
        return result, pipe, y_pred, saved_model_info
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': np.nan,
            'macro_f1': np.nan,
            'weighted_f1': np.nan,
            'extreme_f1': np.nan,
            'success': False,
            'error': str(e)
        }, None, None, None

# ================================================================================================
# 최고 성능 모델 자동 저장 함수
# ================================================================================================

def save_best_models_automatically(results_df, centers):
    """최고 성능 모델들을 자동으로 재학습하여 저장"""
    
    successful_results = results_df[results_df['success'] == True].copy()
    
    if len(successful_results) == 0:
        print("저장할 성공한 모델이 없습니다.")
        return
    
    print("\n=== 최고 성능 모델 자동 저장 ===")
    
    saved_models = []
    
    # 센터별, 타입별 최고 성능 모델 저장
    for center in successful_results['center'].unique():
        center_data = successful_results[successful_results['center'] == center]
        
        for model_type in ['regression', 'classification']:
            type_data = center_data[center_data['type'] == model_type]
            
            if len(type_data) == 0:
                continue
            
            # 최고 성능 모델 선택
            if model_type == 'regression':
                best_model = type_data.loc[type_data['r2'].idxmax()]
                metric_name = 'R²'
                metric_value = best_model['r2']
            else:
                best_model = type_data.loc[type_data['macro_f1'].idxmax()]
                metric_name = 'Macro F1'
                metric_value = best_model['macro_f1']
            
            print(f"\n저장 중: {center} - {best_model['model']} ({model_type})")
            print(f"성능: {metric_name}={metric_value:.3f}")
            
            try:
                # 모델 재학습 및 저장
                df = centers[center]
                target_col = "합계_1일후" if model_type == "regression" else "등급_1일후"
                
                X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
                    df, target_col=target_col, model_type=model_type, 
                    test_size=0.2, split_method=best_model['split_method']
                )
                
                # 모델 재구축
                if model_type == "regression":
                    models = build_regression_models()
                    result, pipe, y_pred, saved_info = evaluate_regression_model_with_save(
                        models[best_model['model']], best_model['model'],
                        X_train, X_test, y_train, y_test,
                        feature_names, center, best_model['split_method'], 
                        save_model=True
                    )
                else:
                    models = build_classification_models()
                    result, pipe, y_pred, saved_info = evaluate_classification_model_with_save(
                        models[best_model['model']], best_model['model'],
                        X_train, X_test, y_train, y_test,
                        feature_names, center, best_model['split_method'],
                        save_model=True
                    )
                
                if saved_info:
                    saved_models.append(saved_info['filename'])
                    
            except Exception as e:
                print(f"저장 실패: {e}")
    
    print(f"\n총 {len(saved_models)}개 모델 저장 완료")
    return saved_models

# ================================================================================================
# 사용 예시
# ================================================================================================

def demo_model_usage():
    """모델 저장 및 사용 데모"""
    
    print("=== 모델 저장 및 사용 데모 ===")
    
    # 1. 저장된 모델 목록 확인
    print("\n1. 저장된 모델 목록:")
    models = list_saved_models()
    for i, model in enumerate(models):
        print(f"  {i+1}. {model['center']} - {model['model']} ({model['type']})")
        print(f"     성능: {model['performance']}")
        print(f"     파일명: {model['filename']}")
    
    # 2. 모델 로드 및 예측 (예시)
    if models:
        print(f"\n2. 첫 번째 모델 로드 및 예측 테스트:")
        first_model = models[0]
        
        # 모델 로드
        model_data = load_trained_model(first_model['filename'])
        
        if model_data:
            print("모델 로드 성공!")
            print(f"예상 피처 개수: {len(model_data['feature_names'])}")
            
            # 더미 데이터로 예측 테스트 (실제 사용시에는 실제 데이터 사용)
            dummy_data = pd.DataFrame({
                feature: [0.5] for feature in model_data['feature_names']
            })
            
            result = predict_with_saved_model(model_data, dummy_data)
            if result:
                print(f"예측 결과: {result['predictions']}")
                if 'probabilities' in result:
                    print(f"예측 확률: {result['probabilities']}")

# 센터별 최고 성능 모델 분석을 위한 함수 수정

def perform_detailed_analysis_with_save_by_center(results_df, centers):
    """센터별 최고 성능 모델에 대한 상세 분석 (저장 기능 포함)"""
    print(f"\n{'='*60}")
    print("=== 센터별 최고 성능 모델 상세 분석 ===")
    print(f"{'='*60}")
    
    successful_results = results_df[results_df['success'] == True].copy()
    
    if len(successful_results) == 0:
        print("분석할 성공한 모델이 없습니다.")
        return []
    
    analyzed_models = []
    
    # 센터별로 최고 성능 모델 분석
    for center in successful_results['center'].unique():
        print(f"\n{'='*50}")
        print(f"센터: {center.upper()}")
        print(f"{'='*50}")
        
        center_data = successful_results[successful_results['center'] == center]
        
        # 1. 센터별 회귀 최고 성능 모델
        reg_results = center_data[center_data['type'] == 'regression']
        if len(reg_results) > 0:
            best_reg = reg_results.loc[reg_results['r2'].idxmax()]
            print(f"\n[{center}] 최고 회귀 성능: {best_reg['model']} ({best_reg['split_method']}) R²={best_reg['r2']:.3f}")
            
            # 회귀 모델 재학습 및 분석
            model_info = analyze_best_model_with_save_detailed(best_reg, centers, 'regression')
            if model_info:
                analyzed_models.append(model_info)
        
        # 2. 센터별 분류 최고 성능 모델
        clf_results = center_data[center_data['type'] == 'classification']
        if len(clf_results) > 0:
            best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
            print(f"\n[{center}] 최고 분류 성능: {best_clf['model']} ({best_clf['split_method']}) F1={best_clf['macro_f1']:.3f}")
            
            # 분류 모델 재학습 및 분석
            model_info = analyze_best_model_with_save_detailed(best_clf, centers, 'classification')
            if model_info:
                analyzed_models.append(model_info)
    
    print(f"\n총 {len(analyzed_models)}개 모델 상세 분석 완료")
    return analyzed_models

def analyze_best_model_with_save_detailed(best_result, centers, model_type):
    """개별 최고 성능 모델 상세 분석 (저장 기능 포함)"""
    center_name = best_result['center']
    model_name = best_result['model']
    split_method = best_result['split_method']
    
    print(f"\n분석 중: {center_name} - {model_name} ({model_type})")
    
    try:
        # 데이터 준비
        df = centers[center_name]
        target_col = "합계_1일후" if model_type == "regression" else "등급_1일후"
        
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            df, target_col=target_col, model_type=model_type, 
            test_size=0.2, split_method=split_method
        )
        
        # 모델 재구축 및 학습
        if model_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
            
        model = models[model_name]
        pipe = make_pipeline_unified(model, model_name, model_type)
        pipe.fit(X_train, y_train)
        
        # 상세 분석 수행 (Feature Importance + SHAP + 저장)
        print(f"  Feature Importance & SHAP 분석 진행 중...")
        importance_df, shap_result = comprehensive_model_analysis_with_save(
            pipe, model_name, X_train, X_test, y_train, y_test, 
            feature_names, model_type, center_name
        )
        
        # 모델도 저장
        performance_metrics = {}
        if model_type == "regression":
            y_pred = pipe.predict(X_test)
            performance_metrics = {
                'r2': best_result['r2'],
                'mae': best_result['mae'], 
                'rmse': best_result['rmse'],
                'mape': best_result['mape']
            }
        else:
            performance_metrics = {
                'accuracy': best_result['accuracy'],
                'macro_f1': best_result['macro_f1'],
                'weighted_f1': best_result['weighted_f1'],
                'extreme_f1': best_result['extreme_f1']
            }
        
        # 학습된 모델 저장
        model_info = {'type': model_type}
        filename, files = save_trained_model(
            pipe, model_info, performance_metrics, feature_names,
            center_name, model_name, split_method
        )
        
        return {
            'center': center_name,
            'model': model_name,
            'type': model_type,
            'split_method': split_method,
            'performance': best_result,
            'saved_model_filename': filename,
            'analysis_completed': True
        }
        
    except Exception as e:
        print(f"  분석 실패: {e}")
        return None

# 기존 run_stratified_comparison 함수에서 호출 부분 수정
def run_stratified_comparison_with_center_analysis():
    """전체 센터 Stratified vs 시계열 비교 + 센터별 상세 분석"""
    print("=== Stratified vs 시계열 분할 비교 실험 (센터별 분석) ===")
    
    # 데이터 로드 확인
    try:
        centers = {
            "nanji": nanji,
            "jungnang": jungnang,  
            "seonam": seonam,
            "tancheon": tancheon
        }
        
        print(f"\n데이터 확인:")
        for name, df in centers.items():
            print(f"  {name}: {len(df)}행")
    
    except NameError:
        print("데이터가 로드되지 않았습니다. 먼저 make_features를 실행하세요.")
        return pd.DataFrame()
    
    # 전체 실험 실행
    all_results = []
    
    for center_name, df in centers.items():
        try:
            center_results = comprehensive_evaluation_comparison(center_name, df)
            all_results.extend(center_results)
        except Exception as e:
            print(f"[{center_name}] 실험 실패: {e}")
    
    # 결과 분석
    results_df = pd.DataFrame(all_results)
    
    if len(results_df) > 0:
        # 기본 분석 및 시각화
        analyze_stratified_comparison(results_df)
        
        # 센터별 최고 성능 모델 상세 분석 (수정된 함수 사용)
        analyzed_models = perform_detailed_analysis_with_save_by_center(results_df, centers)
        
        # 포괄적 결과 저장
        saved_files = save_results_comprehensive(results_df, 'stratified_comparison')
        
        # 모델 성능 비교 데이터도 별도 저장
        successful_results = results_df[results_df['success'] == True]
        if len(successful_results) > 0:
            performance_comparison = create_performance_comparison_data(successful_results)
            save_results_comprehensive(
                performance_comparison, 
                'model_performance'
            )
        
        # 분석 결과 요약
        print(f"\n=== 실험 완료 ===")
        print(f"총 {len(results_df)}개 실험 중 {len(successful_results)}개 성공")
        print(f"센터별 상세 분석: {len(analyzed_models)}개 모델")
        print(f"결과 저장 위치: ../data/results/")
        
        # 분석된 모델 목록 출력
        if analyzed_models:
            print(f"\n=== 분석된 모델 목록 ===")
            for model in analyzed_models:
                print(f"  {model['center']} - {model['model']} ({model['type']}) [{model['split_method']}]")
                print(f"    저장된 모델: {model.get('saved_model_filename', 'N/A')}")
    
    return results_df, analyzed_models

# 센터별 분석 결과 요약 함수
def summarize_center_analysis(analyzed_models):
    """센터별 분석 결과 요약"""
    if not analyzed_models:
        print("분석된 모델이 없습니다.")
        return
    
    print(f"\n{'='*60}")
    print("=== 센터별 최고 성능 모델 요약 ===")
    print(f"{'='*60}")
    
    centers = {}
    for model in analyzed_models:
        center = model['center']
        if center not in centers:
            centers[center] = {'regression': None, 'classification': None}
        centers[center][model['type']] = model
    
    for center, models in centers.items():
        print(f"\n[{center.upper()} 센터]")
        
        if models['regression']:
            reg_model = models['regression']
            print(f"  회귀: {reg_model['model']} ({reg_model['split_method']})")
            print(f"       R² = {reg_model['performance']['r2']:.3f}")
        
        if models['classification']:
            clf_model = models['classification']
            print(f"  분류: {clf_model['model']} ({clf_model['split_method']})")
            print(f"       F1 = {clf_model['performance']['macro_f1']:.3f}")

# 실행 예시 함수
def demo_center_wise_analysis():
    """센터별 분석 데모"""
    print("=== 센터별 분석 실행 ===")
    
    # 센터별 분석 실행
    results_df, analyzed_models = run_stratified_comparison_with_center_analysis()
    
    # 결과 요약
    summarize_center_analysis(analyzed_models)
    
    return results_df, analyzed_models

# ================================================================================================
# 9. 실행 가이드 및 메인 실행 부분
# ================================================================================================
if __name__ == "__main__":
    print("=== 완성된 Stratified vs 시계열 분할 비교 실험 ===")
    
    print("\n📁 결과 저장 구조:")
    print("../data/results/")
    print("├── stratified_comparison/  # 분할 방법 비교 결과")
    print("│   ├── stratified_comparison_YYYYMMDD_HHMMSS.csv")
    print("│   ├── stratified_summary_YYYYMMDD_HHMMSS.csv")
    print("│   └── stratified_best_models_YYYYMMDD_HHMMSS.csv")
    print("├── feature_importance/     # 피처 중요도 분석")
    print("│   └── importance_센터명_모델명_YYYYMMDD_HHMMSS.csv")
    print("├── shap_analysis/         # SHAP 분석 결과")
    print("│   ├── shap_values_센터명_모델명_YYYYMMDD_HHMMSS.pkl")
    print("│   └── shap_summary_센터명_모델명_YYYYMMDD_HHMMSS.csv")
    print("├── model_performance/     # 모델 성능 상세 분석")
    print("│   └── performance_YYYYMMDD_HHMMSS.csv")
    print("└── visualizations/        # 생성된 그래프 이미지")
    print("    ├── stratified_comparison_plots_YYYYMMDD_HHMMSS.png")
    print("    ├── feature_importance_센터명_모델명_YYYYMMDD_HHMMSS.png")
    print("    └── shap_센터명_모델명_YYYYMMDD_HHMMSS.png")
    
    print("\n🚀 실행 방법:")
    print("1. 전체 실험 (권장): ")
    print("   results_df = run_stratified_comparison()")
    print("\n2. 단일 센터 테스트:")
    print("   results_df = quick_stratified_test('nanji')")
    print("\n3. 결과 분석만:")
    print("   analyze_stratified_comparison(results_df)")
    
    print(f"\n💡 주요 개선사항:")
    print("- CatBoost 회귀 모델 오류 수정")
    print("- CatBoost 분류 모델 파라미터 오류 수정")
    print("- 체계적인 폴더 구조로 결과 저장")
    print("- Feature Importance & SHAP 분석 결과 저장")
    print("- 시각화 이미지 자동 저장")
    print("- 요약 통계 및 최고 성능 모델 별도 저장")
    print("- 모델별 상세 분석 결과 추적 가능")
    
    print(f"\n📊 실험 구성:")
    print("- 4개 센터 (nanji, jungnang, seonam, tancheon)")
    print("- 2가지 분할방법 (temporal, stratified)")
    print("- 12개 모델 (회귀 6개 + 분류 6개)")
    print("- 총 96개 실험 (4×2×12)")
    
    print(f"\n📈 평가 지표:")
    print("회귀: R², MAE, RMSE, MAPE")
    print("분류: Accuracy, Macro F1, Weighted F1, Extreme F1")
    
    print(f"\n⚠️ 사용 전 확인사항:")
    print("1. 데이터 준비: nanji, jungnang, seonam, tancheon 변수가 로드되어 있어야 함")
    print("2. 필요 라이브러리: pandas, numpy, matplotlib, sklearn 등")
    print("3. 선택 라이브러리: xgboost, lightgbm, catboost, shap")
    print("4. 실행 권한: ../data/results/ 폴더 생성 권한 필요")

=== 완성된 Stratified vs 시계열 분할 비교 실험 ===

📁 결과 저장 구조:
../data/results/
├── stratified_comparison/  # 분할 방법 비교 결과
│   ├── stratified_comparison_YYYYMMDD_HHMMSS.csv
│   ├── stratified_summary_YYYYMMDD_HHMMSS.csv
│   └── stratified_best_models_YYYYMMDD_HHMMSS.csv
├── feature_importance/     # 피처 중요도 분석
│   └── importance_센터명_모델명_YYYYMMDD_HHMMSS.csv
├── shap_analysis/         # SHAP 분석 결과
│   ├── shap_values_센터명_모델명_YYYYMMDD_HHMMSS.pkl
│   └── shap_summary_센터명_모델명_YYYYMMDD_HHMMSS.csv
├── model_performance/     # 모델 성능 상세 분석
│   └── performance_YYYYMMDD_HHMMSS.csv
└── visualizations/        # 생성된 그래프 이미지
    ├── stratified_comparison_plots_YYYYMMDD_HHMMSS.png
    ├── feature_importance_센터명_모델명_YYYYMMDD_HHMMSS.png
    └── shap_센터명_모델명_YYYYMMDD_HHMMSS.png

🚀 실행 방법:
1. 전체 실험 (권장): 
   results_df = run_stratified_comparison()

2. 단일 센터 테스트:
   results_df = quick_stratified_test('nanji')

3. 결과 분석만:
   analyze_stratified_comparison(results_df)

💡 주요 개선사항:
- CatBoost 회귀 모델 오류 수정
- CatBoost 분류 모델 파라

In [58]:
# 사용 예시
"""
# 센터별 최고 성능 모델 분석 실행
results_df, analyzed_models = run_stratified_comparison_with_center_analysis()

# 분석 결과 요약
summarize_center_analysis(analyzed_models)

# 저장된 모델 목록 확인
models = list_saved_models()
for model in models:
    print(f"{model['center']} - {model['model']} ({model['type']}): {model['performance']}")
"""

# 실행 예시
"""
# 1. 실험 실행 후 최고 성능 모델들 자동 저장
results_df = run_stratified_comparison()
saved_models = save_best_models_automatically(results_df, centers)

# 2. 저장된 모델 확인
demo_model_usage()

# 3. 새로운 데이터에 예측
model_data = load_trained_model('nanji_XGBoost_Reg_temporal_20250826_143022')
new_predictions = predict_with_saved_model(model_data, new_dataframe)
"""

# ================================================================================================
# 10. 사용 예시
# ================================================================================================
"""
실제 사용 예시:

# 1. 전체 실험 실행
results_df = run_stratified_comparison()

# 2. 결과 확인
print(f"총 {len(results_df)}개 실험 완료")
successful = results_df[results_df['success'] == True]
print(f"성공: {len(successful)}개")

# 3. 최고 성능 모델 확인
reg_best = successful[successful['type'] == 'regression'].nlargest(1, 'r2')
clf_best = successful[successful['type'] == 'classification'].nlargest(1, 'macro_f1')

print("최고 회귀 성능:", reg_best[['center', 'model', 'split_method', 'r2']].iloc[0])
print("최고 분류 성능:", clf_best[['center', 'model', 'split_method', 'macro_f1']].iloc[0])

# 4. 저장된 파일 확인
import os
for root, dirs, files in os.walk('../data/results'):
    level = root.replace('../data/results', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f'{subindent}{file}')
"""

'\n실제 사용 예시:\n\n# 1. 전체 실험 실행\nresults_df = run_stratified_comparison()\n\n# 2. 결과 확인\nprint(f"총 {len(results_df)}개 실험 완료")\nsuccessful = results_df[results_df[\'success\'] == True]\nprint(f"성공: {len(successful)}개")\n\n# 3. 최고 성능 모델 확인\nreg_best = successful[successful[\'type\'] == \'regression\'].nlargest(1, \'r2\')\nclf_best = successful[successful[\'type\'] == \'classification\'].nlargest(1, \'macro_f1\')\n\nprint("최고 회귀 성능:", reg_best[[\'center\', \'model\', \'split_method\', \'r2\']].iloc[0])\nprint("최고 분류 성능:", clf_best[[\'center\', \'model\', \'split_method\', \'macro_f1\']].iloc[0])\n\n# 4. 저장된 파일 확인\nimport os\nfor root, dirs, files in os.walk(\'../data/results\'):\n    level = root.replace(\'../data/results\', \'\').count(os.sep)\n    indent = \' \' * 2 * level\n    print(f\'{indent}{os.path.basename(root)}/\')\n    subindent = \' \' * 2 * (level + 1)\n    for file in files:\n        print(f\'{subindent}{file}\')\n'

In [59]:
# ================================================================================================
# 완전한 운영 환경 시뮬레이션 파이프라인
# 1단계: 5월 20일까지 데이터로 모델 학습 및 성능 평가
# 2단계: 최고 성능 모델 선택 및 저장
# 3단계: 5월 21일~31일 데이터로 예측 수행
# 4단계: 결과 테이블 생성 및 성능 평가
# ================================================================================================

# 기존 함수들 활용
def complete_production_simulation_pipeline(centers=None, cutoff_date='2025-05-20'):
    """
    완전한 운영 환경 시뮬레이션 파이프라인
    
    Parameters:
    - centers: 센터별 데이터 딕셔너리
    - cutoff_date: 학습/예측 분할 기준일
    
    Returns:
    - final_results_table: 최종 예측 결과 테이블
    - trained_models_info: 학습된 모델 정보
    - performance_summary: 성능 요약
    """
    
    print(f"{'='*80}")
    print(f"완전한 운영 환경 시뮬레이션 파이프라인 시작")
    print(f"학습 기간: ~ {cutoff_date}")
    print(f"예측 기간: {cutoff_date} 이후")
    print(f"{'='*80}")
    
    # 데이터 로드 확인
    if centers is None:
        try:
            centers = {
                "nanji": nanji,
                "jungnang": jungnang,  
                "seonam": seonam,
                "tancheon": tancheon
            }
            print(f"데이터 로드 완료:")
            for name, df in centers.items():
                print(f"  {name}: {len(df)}행")
        except NameError:
            print("데이터가 로드되지 않았습니다. 먼저 데이터를 로드하세요.")
            return None
    
    cutoff = pd.to_datetime(cutoff_date)
    
    # ========================================================================================
    # 1단계: 각 센터별로 5월 20일까지 데이터로 모델 학습 및 성능 평가
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"1단계: 모델 학습 및 성능 평가 (~ {cutoff_date})")
    print(f"{'='*60}")
    
    all_training_results = []
    best_models_by_center = {}
    
    for center_name, df in centers.items():
        print(f"\n[{center_name.upper()} 센터 처리 중...]")
        
        # 날짜 변환 및 데이터 분할
        df_work = df.copy()
        df_work['날짜'] = pd.to_datetime(df_work['날짜'])
        
        train_data = df_work[df_work['날짜'] <= cutoff].copy()
        future_data = df_work[df_work['날짜'] > cutoff].copy()
        
        print(f"  학습 데이터: {len(train_data)}행")
        print(f"  예측 데이터: {len(future_data)}행")
        
        if len(train_data) < 50:
            print(f"  학습 데이터가 부족합니다. 건너뜁니다.")
            continue
            
        if len(future_data) == 0:
            print(f"  예측할 데이터가 없습니다. 건너뜁니다.")
            continue
        
        # 모델 성능 평가 (기존 함수 활용)
        center_results = comprehensive_evaluation_comparison(center_name, train_data)
        all_training_results.extend(center_results)
        
        # 최고 성능 모델 선택
        center_best_models = select_and_train_best_models(center_name, train_data, center_results)
        if center_best_models:
            best_models_by_center[center_name] = center_best_models
    
    # ========================================================================================
    # 2단계: 최고 성능 모델 선택 결과 요약
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"2단계: 최고 성능 모델 선택 완료")
    print(f"{'='*60}")
    
    training_results_df = pd.DataFrame(all_training_results)
    
    for center, models in best_models_by_center.items():
        print(f"\n[{center.upper()} 센터 최고 성능 모델]")
        if 'regression' in models:
            reg_info = models['regression']
            print(f"  회귀: {reg_info['model_name']} (R²={reg_info['performance']['r2']:.3f})")
        if 'classification' in models:
            clf_info = models['classification']
            print(f"  분류: {clf_info['model_name']} (F1={clf_info['performance']['macro_f1']:.3f})")
    
    # ========================================================================================
    # 3단계: 5월 21일~31일 데이터로 예측 수행
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"3단계: 새로운 데이터 예측 수행 ({cutoff_date} 이후)")
    print(f"{'='*60}")
    
    all_predictions = []
    
    for center_name, df in centers.items():
        if center_name not in best_models_by_center:
            continue
            
        print(f"\n[{center_name.upper()} 센터 예측 중...]")
        
        # 데이터 준비
        df_work = df.copy()
        df_work['날짜'] = pd.to_datetime(df_work['날짜'])
        future_data = df_work[df_work['날짜'] > cutoff].copy()
        
        if len(future_data) == 0:
            continue
        
        # 예측 수행
        center_predictions = make_predictions_for_center(
            center_name, future_data, best_models_by_center[center_name]
        )
        all_predictions.extend(center_predictions)
    
    # ========================================================================================
    # 4단계: 결과 테이블 생성 및 성능 평가
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"4단계: 결과 테이블 생성 및 성능 평가")
    print(f"{'='*60}")
    
    if not all_predictions:
        print("예측 결과가 없습니다.")
        return None
    
    # 최종 결과 테이블 생성
    final_results_table = create_final_results_table(all_predictions)
    
    # 성능 요약 생성
    performance_summary = create_performance_summary(final_results_table)
    
    # 결과 출력
    print_final_results(final_results_table, performance_summary)
    
    # 결과 저장
    save_final_results(final_results_table, performance_summary, training_results_df)
    
    return final_results_table, best_models_by_center, performance_summary

def select_and_train_best_models(center_name, train_data, evaluation_results):
    """최고 성능 모델 선택 및 학습"""
    
    results_df = pd.DataFrame(evaluation_results)
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) == 0:
        print(f"    성공한 모델이 없습니다.")
        return None
    
    best_models = {}
    
    # 회귀 최고 성능 모델
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"    최고 회귀 모델: {best_reg['model']} (R²={best_reg['r2']:.3f})")
        
        # 모델 재학습
        reg_pipeline = retrain_best_model(
            train_data, best_reg['model'], 'regression', best_reg['split_method']
        )
        
        if reg_pipeline:
            best_models['regression'] = {
                'model_name': best_reg['model'],
                'pipeline': reg_pipeline['pipeline'],
                'feature_names': reg_pipeline['feature_names'],
                'performance': dict(best_reg),
                'split_method': best_reg['split_method']
            }
    
    # 분류 최고 성능 모델
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"    최고 분류 모델: {best_clf['model']} (F1={best_clf['macro_f1']:.3f})")
        
        # 모델 재학습
        clf_pipeline = retrain_best_model(
            train_data, best_clf['model'], 'classification', best_clf['split_method']
        )
        
        if clf_pipeline:
            best_models['classification'] = {
                'model_name': best_clf['model'],
                'pipeline': clf_pipeline['pipeline'],
                'feature_names': clf_pipeline['feature_names'],
                'performance': dict(best_clf),
                'split_method': best_clf['split_method']
            }
    
    return best_models if best_models else None

def retrain_best_model(train_data, model_name, model_type, split_method):
    """최고 성능 모델 재학습"""
    
    try:
        target_col = "합계_1일후" if model_type == "regression" else "등급_1일후"
        
        # 전체 학습 데이터로 재학습 (test_size를 매우 작게 설정)
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=model_type, 
            test_size=0.05, split_method=split_method
        )
        
        # 전체 데이터 사용 (X_train + X_test)
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        X_all = X_all[feature_names]  # ✅ 피처 순서 고정
        
        # 모델 구축 및 학습
        if model_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, model_type)
        pipeline.fit(X_all, y_all)
        
        return {
            'pipeline': pipeline,
            'feature_names': feature_names
        }
        
    except Exception as e:
        print(f"    모델 재학습 실패 ({model_name}): {e}")
        return None

def make_predictions_for_center(center_name, future_data, trained_models):
    """센터별 예측 수행"""
    predictions = []
    future_data = future_data.reset_index(drop=True)  # make_predictions_for_center 진입 직후

    for task_type, model_info in trained_models.items():
        try:
            pipeline = model_info['pipeline']
            feature_names = model_info['feature_names']
            model_name = model_info['model_name']
            
            # 타겟 컬럼 설정
            target_col = "합계_1일후" if task_type == "regression" else "등급_1일후"
            
            # 예측 데이터 준비
            X_future, y_true = prepare_prediction_data(future_data, feature_names, target_col)
            
            if X_future is None or len(X_future) == 0:
                print(f"    {task_type} 예측 데이터 준비 실패")
                continue
            
            # 예측 수행
            y_pred = pipeline.predict(X_future)
            # 타입 보정
            if task_type == "classification":
                # numpy 타입 포함 -> 파이썬 int로
                y_pred = [int(v) for v in y_pred]
            
            print(f"    {task_type} 예측 완료: {len(y_pred)}개")
            
            # 결과 저장
            for i in range(len(X_future)):
                actual_val = (y_true.iloc[i] if (y_true is not None and i < len(y_true) and not pd.isna(y_true.iloc[i])) else None)
                
                pred_result = {
                    'date': future_data.iloc[i]['날짜'],
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_name,
                    'target_column': target_col,
                    # 'actual_value': y_true.iloc[i] if i < len(y_true) and not pd.isna(y_true.iloc[i]) else None,
                    'actual_value': actual_val,
                    # 'predicted_value': float(y_pred[i])
                    'predicted_value': int(y_pred[i]) if task_type=='classification' else float(y_pred[i])
                }
                predictions.append(pred_result)
                
        except Exception as e:
            print(f"    {task_type} 예측 실패: {e}")
    
    return predictions

def prepare_prediction_data(future_data, expected_features, target_col):
    """예측용 데이터 전처리"""
    
    # 제외할 컬럼들
    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    # 피처 데이터 준비
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in future_data.columns]
    X_future = future_data.drop(columns=drop_cols, errors="ignore")
    
    # 수치형 변환
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    # 실제값 추출
    y_true = None
    if target_col in future_data.columns:
        if target_col == "등급_1일후":
            y_true = pd.to_numeric(future_data[target_col], errors="coerce").astype("Int64")
        else:
            y_true = pd.to_numeric(future_data[target_col], errors="coerce")
    
    # 피처 순서 맞춤 및 누락 피처 처리
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    # 피처 순서 맞춤
    X_future = X_future[expected_features].copy()
    
    return X_future, y_true

def create_final_results_table(all_predictions):
    """최종 결과 테이블 생성"""
    
    results_df = pd.DataFrame(all_predictions)
    
    # 날짜 정렬
    results_df = results_df.sort_values(['date', 'center', 'task_type'])
    
    # 평가 지표 계산
    results_df = calculate_prediction_metrics_enhanced(results_df)
    
    return results_df

def calculate_prediction_metrics_enhanced(results_df):
    """향상된 예측 평가 지표 계산"""
    
    results_df = results_df.copy()
    
    # 평가 지표 컬럼 초기화
    results_df['absolute_error'] = None
    results_df['squared_error'] = None
    results_df['percentage_error'] = None
    results_df['correct_prediction'] = None
    results_df['residual'] = None
    
    for idx, row in results_df.iterrows():
        if pd.isna(row['actual_value']) or pd.isna(row['predicted_value']):
            continue
            
        actual = row['actual_value']
        predicted = row['predicted_value']
        
        if row['task_type'] == 'regression':
            # 회귀 평가 지표
            residual = actual - predicted
            abs_error = abs(residual)
            sq_error = residual ** 2
            pct_error = abs(residual) / (abs(actual) + 1e-8) * 100
            
            results_df.at[idx, 'residual'] = residual
            results_df.at[idx, 'absolute_error'] = abs_error
            results_df.at[idx, 'squared_error'] = sq_error
            results_df.at[idx, 'percentage_error'] = pct_error
            
        else:  # classification
            # 분류 평가 지표
            correct = 1 if int(actual) == int(predicted) else 0
            results_df.at[idx, 'correct_prediction'] = correct
    
    return results_df

def create_performance_summary(results_df):
    """성능 요약 생성"""
    
    summary = {}
    
    for center in results_df['center'].unique():
        center_data = results_df[results_df['center'] == center]
        summary[center] = {}
        
        for task_type in ['regression', 'classification']:
            task_data = center_data[center_data['task_type'] == task_type]
            task_data_clean = task_data.dropna(subset=['actual_value', 'predicted_value'])
            
            if len(task_data_clean) > 0:
                if task_type == 'regression':
                    summary[center]['regression'] = {
                        'model_name': task_data_clean.iloc[0]['model_name'],
                        'prediction_count': len(task_data_clean),
                        'mae': task_data_clean['absolute_error'].mean(),
                        'rmse': np.sqrt(task_data_clean['squared_error'].mean()),
                        'mape': task_data_clean['percentage_error'].mean(),
                        'r2_on_predictions': calculate_r2_on_predictions(
                            task_data_clean['actual_value'], 
                            task_data_clean['predicted_value']
                        )
                    }
                else:
                    summary[center]['classification'] = {
                        'model_name': task_data_clean.iloc[0]['model_name'],
                        'prediction_count': len(task_data_clean),
                        'accuracy': task_data_clean['correct_prediction'].mean(),
                        'correct_count': int(task_data_clean['correct_prediction'].sum()),
                        'total_count': len(task_data_clean)
                    }
    
    return summary

def calculate_r2_on_predictions(y_true, y_pred):
    """예측값에 대한 R² 계산"""
    from sklearn.metrics import r2_score
    y_true = pd.Series(y_true).astype(float)
    y_pred = pd.Series(y_pred).astype(float)
    if len(y_true) < 2:
        return None
    return r2_score(y_true, y_pred)


def print_final_results(results_df, performance_summary):
    """최종 결과 출력"""
    
    print(f"\n{'='*60}")
    print(f"=== 최종 예측 결과 요약 ===")
    print(f"{'='*60}")
    
    # 전체 요약
    total_predictions = len(results_df)
    date_range = f"{results_df['date'].min()} ~ {results_df['date'].max()}"
    centers = results_df['center'].unique()
    
    print(f"예측 기간: {date_range}")
    print(f"총 예측 건수: {total_predictions}")
    print(f"센터 수: {len(centers)} ({', '.join(centers)})")
    
    # 센터별 성능 요약
    for center, perf in performance_summary.items():
        print(f"\n--- {center.upper()} 센터 성능 ---")
        
        if 'regression' in perf:
            reg = perf['regression']
            print(f"  회귀 ({reg['model_name']}):")
            print(f"    예측 건수: {reg['prediction_count']}")
            print(f"    MAE: {reg['mae']:.2f}")
            print(f"    RMSE: {reg['rmse']:.2f}")
            print(f"    MAPE: {reg['mape']:.1f}%")
            if reg['r2_on_predictions'] is not None:
                print(f"    R²: {reg['r2_on_predictions']:.3f}")
        
        if 'classification' in perf:
            clf = perf['classification']
            print(f"  분류 ({clf['model_name']}):")
            print(f"    예측 건수: {clf['prediction_count']}")
            print(f"    정확도: {clf['accuracy']:.1%}")
            print(f"    정답 개수: {clf['correct_count']}/{clf['total_count']}")
    
    # 결과 테이블 미리보기
    print(f"\n--- 결과 테이블 미리보기 ---")
    display_columns = ['date', 'center', 'task_type', 'model_name', 'actual_value', 'predicted_value']
    if all(col in results_df.columns for col in display_columns):
        print(results_df[display_columns].head(10).to_string(index=False))

def save_final_results(results_df, performance_summary, training_results_df):
    """최종 결과 저장"""
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    base_filename = f"production_simulation_{timestamp}"
    
    try:
        # 1. 예측 결과 테이블 저장
        results_filename = f"{base_filename}_predictions.csv"
        results_df.to_csv(results_filename, index=False, encoding='utf-8-sig')
        print(f"\n예측 결과 저장: {results_filename}")
        
        # 2. 성능 요약 저장
        summary_data = []
        for center, perf in performance_summary.items():
            for task_type, metrics in perf.items():
                summary_row = {'center': center, 'task_type': task_type}
                summary_row.update(metrics)
                summary_data.append(summary_row)
        
        if summary_data:
            summary_df = pd.DataFrame(summary_data)
            summary_filename = f"{base_filename}_summary.csv"
            summary_df.to_csv(summary_filename, index=False, encoding='utf-8-sig')
            print(f"성능 요약 저장: {summary_filename}")
        
        # 3. 학습 결과 저장
        if len(training_results_df) > 0:
            training_filename = f"{base_filename}_training.csv"
            training_results_df.to_csv(training_filename, index=False, encoding='utf-8-sig')
            print(f"학습 결과 저장: {training_filename}")
        
        print(f"\n모든 결과가 저장되었습니다. 파일명 접두사: {base_filename}")
        
    except Exception as e:
        print(f"결과 저장 중 오류 발생: {e}")

# ================================================================================================
# 메인 실행 함수
# ================================================================================================

# ================================================================================================
# 메인 실행 함수 (✅ 수정된 버전)
# ================================================================================================

def run_complete_production_pipeline(cutoff_date='2025-05-20'):
    """완전한 파이프라인 실행"""
    
    print("완전한 운영 환경 시뮬레이션 파이프라인을 시작합니다...")
    print("이 과정은 다소 시간이 걸릴 수 있습니다.")
    
    start_time = time.time()
    
    # 데이터 로드 (이 부분은 실행기에서 직접 처리하는 것이 더 명확합니다)
    try:
        nanji_raw = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')
        jungnang_raw = pd.read_csv('../data/processed/center_season/jungnang/중랑_merged.csv', encoding='utf-8-sig')
        seonam_raw = pd.read_csv('../data/processed/center_season/seonam/서남_merged.csv', encoding='utf-8-sig')
        tancheon_raw = pd.read_csv('../data/processed/center_season/tancheon/탄천_merged.csv', encoding='utf-8-sig')
        
        centers = {
            "nanji": nanji_raw,
            "jungnang": jungnang_raw,
            "seonam": seonam_raw,
            "tancheon": tancheon_raw
        }
    except Exception as e:
        print(f"원본 데이터 파일 로드 실패: {e}")
        return None

    # 핵심 파이프라인 함수 호출 (자기 자신이 아닌!)
    results = complete_production_simulation_pipeline(centers=centers, cutoff_date=cutoff_date)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\n{'='*60}")
    print(f"파이프라인 실행 완료!")
    print(f"총 소요시간: {elapsed_time:.1f}초 ({elapsed_time/60:.1f}분)")
    print(f"{'='*60}")
    
    return results

# ================================================================================================
# 사용 예시
# ================================================================================================

if __name__ == "__main__":
    print("=== 완전한 운영 환경 시뮬레이션 파이프라인 ===")
    print()
    print("사용법:")
    print("results = run_complete_production_pipeline()")
    print()
    print("또는 cutoff_date를 변경하여:")
    print("results = run_complete_production_pipeline(cutoff_date='2025-05-15')")
    print()
    print("반환값:")
    print("- results[0]: 최종 예측 결과 테이블")
    print("- results[1]: 학습된 모델 정보")  
    print("- results[2]: 성능 요약")
    print()
    print("생성되는 파일:")
    print("- production_simulation_YYYYMMDD_HHMMSS_predictions.csv")
    print("- production_simulation_YYYYMMDD_HHMMSS_summary.csv")
    print("- production_simulation_YYYYMMDD_HHMMSS_training.csv")

# 완전한 파이프라인 실행
results = run_complete_production_pipeline(cutoff_date='2025-05-20')

# 결과 확인
final_table, trained_models, performance_summary = results
print(final_table.head())

=== 완전한 운영 환경 시뮬레이션 파이프라인 ===

사용법:
results = run_complete_production_pipeline()

또는 cutoff_date를 변경하여:
results = run_complete_production_pipeline(cutoff_date='2025-05-15')

반환값:
- results[0]: 최종 예측 결과 테이블
- results[1]: 학습된 모델 정보
- results[2]: 성능 요약

생성되는 파일:
- production_simulation_YYYYMMDD_HHMMSS_predictions.csv
- production_simulation_YYYYMMDD_HHMMSS_summary.csv
- production_simulation_YYYYMMDD_HHMMSS_training.csv
완전한 운영 환경 시뮬레이션 파이프라인을 시작합니다...
이 과정은 다소 시간이 걸릴 수 있습니다.
완전한 운영 환경 시뮬레이션 파이프라인 시작
학습 기간: ~ 2025-05-20
예측 기간: 2025-05-20 이후

1단계: 모델 학습 및 성능 평가 (~ 2025-05-20)

[NANJI 센터 처리 중...]
  학습 데이터: 3062행
  예측 데이터: 41행

센터: nanji - Stratified vs 시계열 분할 비교
데이터 크기: 3062행, 28컬럼

분할 방법: TEMPORAL

--- 회귀 모델 평가 (temporal) ---
회귀 모델 평가 실패 (temporal): '합계_1일후'

--- 분류 모델 평가 (temporal) ---
분류 모델 평가 실패 (temporal): '등급_1일후'

분할 방법: STRATIFIED

--- 회귀 모델 평가 (random_shuffle) ---
회귀 모델 평가 실패 (random_shuffle): '합계_1일후'

--- 분류 모델 평가 (stratified) ---
분류 모델 평가 실패 (stratified): '등급_1일후'


KeyError: 'success'

In [ ]:
# ================================================================================================
# 1. 수정된 make_features 함수
# ================================================================================================

def make_features(df, cutoff_date=None):
    """
    파생변수 생성 함수 - Data Leakage 방지 버전
    
    Parameters:
    - df: 원본 데이터
    - cutoff_date: 타겟 변수 생성 제한 날짜 (None이면 전체 데이터 사용)
    """
    df = df.copy()
    
    # 날짜 정리 및 정렬
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)

    # 달/요일 숫자
    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday

    # 계절/불쾌지수등급 숫자 매핑
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    df['계절'] = df['계절'].map(season_map).astype('Int64')
    df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')

    # 강수량 시차 피처
    df['강수량_1일전'] = df['일_일강수량(mm)'].shift(1)
    df['강수량_2일전'] = df['일_일강수량(mm)'].shift(2)
    df['강수량_1일_누적'] = df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    df['강수량_2일_누적'] = df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    df['강수량_3일_누적'] = df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    df['강수량_5일_누적'] = df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    df['강수량_7일_누적'] = df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()

    df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    if '일_평균기온(°C)' in df.columns:
        T = pd.to_numeric(df['일_평균기온(°C)'], errors='coerce')
    else:
        T = pd.Series(np.nan, index=df.index)
    if '일_평균풍속(m/s)' in df.columns:
        V_ms = pd.to_numeric(df['일_평균풍속(m/s)'], errors='coerce')
    else:
        V_ms = pd.Series(np.nan, index=df.index)
    if '평균습도(%)' in df.columns:
        RH = pd.to_numeric(df['평균습도(%)'], errors='coerce')
    else:
        RH = pd.Series(np.nan, index=df.index)

    # 윈드칠
    V_kmh = V_ms * 3.6
    wct_raw = 13.12 + 0.6215*T - 11.37*np.power(V_kmh, 0.16) + 0.3965*T*np.power(V_kmh, 0.16)
    wc_valid = (T <= 10.0) & (V_kmh >= 4.8)
    wct = T.copy()
    wct[wc_valid] = wct_raw[wc_valid]

    # 열지수
    T_f = T * 9/5 + 32
    HI_f = (-42.379 + 2.04901523*T_f + 10.14333127*RH
            - 0.22475541*T_f*RH - 0.00683783*T_f**2 - 0.05481717*RH**2
            + 0.00122874*T_f**2*RH + 0.00085282*T_f*RH**2
            - 0.00000199*T_f**2*RH**2)
    mask_low = (RH < 13) & (T_f >= 80) & (T_f <= 112)
    adj_low = ((13 - RH)/4) * np.sqrt((17 - np.abs(T_f - 95))/17)
    HI_f = HI_f.where(~mask_low, HI_f - adj_low)
    mask_high = (RH > 85) & (T_f >= 80) & (T_f <= 87)
    adj_high = ((RH - 85)/10) * ((87 - T_f)/5)
    HI_f = HI_f.where(~mask_high, HI_f + adj_high)
    hi_valid = (T_f >= 80) & (RH >= 40)
    HI_c = (HI_f - 32) * 5/9
    hi = T.copy()
    hi[hi_valid] = HI_c[hi_valid]

    # 스테드먼 체감온도
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    at = T + 0.33*e - 0.70*V_ms - 4.00

    # 최종 체감온도
    apparent = at.copy()
    apparent[hi_valid] = hi[hi_valid]
    apparent[wc_valid] = wct[wc_valid]
    df['체감온도(°C)'] = apparent
    
    # 분류용 등급 계산
    q = df['합계'].dropna().quantile([0.15, 0.70, 0.90])
    q15, q70, q90 = float(q.loc[0.15]), float(q.loc[0.70]), float(q.loc[0.90])

    def categorize(x):
        if pd.isna(x):
            return np.nan
        if x < q15:
            return 0
        elif x < q70:
            return 1
        elif x < q90:
            return 2
        else:
            return 3

    df['등급'] = df['합계'].apply(categorize)
    
    # 타겟 변수 생성 (핵심 수정 부분)
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        
        # 타겟 변수 초기화
        df['합계_1일후'] = np.nan
        df['합계_2일후'] = np.nan
        df['등급_1일후'] = np.nan
        df['등급_2일후'] = np.nan
        
        # cutoff_date 내에서만 타겟 변수 생성
        for i in range(len(df)):
            current_date = df.loc[i, '날짜']
            
            if i + 1 < len(df) and current_date <= cutoff:
                next_date = df.loc[i+1, '날짜']
                if next_date <= cutoff:
                    df.loc[i, '합계_1일후'] = df.loc[i+1, '합계']
                    df.loc[i, '등급_1일후'] = df.loc[i+1, '등급']
            
            if i + 2 < len(df) and current_date <= cutoff:
                next2_date = df.loc[i+2, '날짜']
                if next2_date <= cutoff:
                    df.loc[i, '합계_2일후'] = df.loc[i+2, '합계']
                    df.loc[i, '등급_2일후'] = df.loc[i+2, '등급']
    else:
        # 기존 방식
        df['합계_1일후'] = df['합계'].shift(-1)
        df['합계_2일후'] = df['합계'].shift(-2)
        df['등급_1일후'] = df['등급'].shift(-1).astype('Int64')
        df['등급_2일후'] = df['등급'].shift(-2).astype('Int64')

    # 컷 기준 저장
    df.attrs['cutoffs'] = {"q15": q15, "q70": q70, "q90": q90}

    # 결측 제거 및 리셋
    df = df.dropna().reset_index(drop=True)
    
    # 6월 데이터 제거
    df = df[df["날짜"] < "2025-06-01"]
    
    return df

# ================================================================================================
# 2. 예측용 파생변수 생성 함수
# ================================================================================================

def make_features_for_prediction(historical_df, future_df):
    """새로운 데이터에 대한 파생변수 생성 (과거 데이터 활용)"""
    
    # 전체 데이터 결합
    combined_df = pd.concat([historical_df, future_df], ignore_index=True)
    combined_df['날짜'] = pd.to_datetime(combined_df['날짜'])
    combined_df = combined_df.sort_values('날짜').reset_index(drop=True)
    
    # 기본 파생변수들만 생성 (lag 변수 중심)
    combined_df['월'] = combined_df['날짜'].dt.month
    combined_df['요일'] = combined_df['날짜'].dt.weekday
    
    # 계절/불쾌지수 매핑
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    combined_df['계절'] = combined_df['계절'].map(season_map).astype('Int64')
    combined_df['불쾌지수등급'] = combined_df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 시차 변수들
    combined_df['강수량_1일전'] = combined_df['일_일강수량(mm)'].shift(1)
    combined_df['강수량_2일전'] = combined_df['일_일강수량(mm)'].shift(2)
    combined_df['강수량_1일_누적'] = combined_df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    combined_df['강수량_2일_누적'] = combined_df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    combined_df['강수량_3일_누적'] = combined_df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    combined_df['강수량_5일_누적'] = combined_df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    combined_df['강수량_7일_누적'] = combined_df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()
    
    combined_df['일교차'] = combined_df['일_최고기온(°C)'] - combined_df['일_최저기온(°C)']
    combined_df['폭우_여부'] = (combined_df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산 (동일한 로직)
    T = pd.to_numeric(combined_df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(combined_df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(combined_df.get('평균습도(%)', np.nan), errors='coerce')
    
    # 체감온도 계산 (간단 버전)
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    combined_df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 새 데이터 부분만 반환
    historical_len = len(historical_df)
    return combined_df.iloc[historical_len:].reset_index(drop=True)

# ================================================================================================
# 3. 수정된 메인 파이프라인
# ================================================================================================

def complete_production_simulation_safe(centers=None, cutoff_date='2025-05-20'):
    """Data Leakage 방지 완전 파이프라인"""
    
    print(f"{'='*80}")
    print(f"Data Leakage 방지 운영 환경 시뮬레이션")
    print(f"학습 기간: ~ {cutoff_date}")
    print(f"예측 기간: {cutoff_date} 이후")
    print(f"{'='*80}")
    
    if centers is None:
        try:
            centers = load_original_data()
        except:
            print("데이터를 로드할 수 없습니다.")
            return None
    
    cutoff = pd.to_datetime(cutoff_date)
    all_training_results = []
    best_models_by_center = {}
    all_predictions = []
    
    # 각 센터별 처리
    for center_name, df_raw in centers.items():
        print(f"\n[{center_name.upper()} 센터 처리]")
        
        # 원본 데이터 날짜 기준 분할
        df_raw['날짜'] = pd.to_datetime(df_raw['날짜'])
        df_raw = df_raw.sort_values('날짜').reset_index(drop=True)
        
        raw_train_data = df_raw[df_raw['날짜'] <= cutoff].copy()
        raw_future_data = df_raw[df_raw['날짜'] > cutoff].copy()
        
        print(f"  원본 학습 데이터: {len(raw_train_data)}행")
        print(f"  원본 예측 데이터: {len(raw_future_data)}행")
        
        if len(raw_train_data) < 50 or len(raw_future_data) == 0:
            print(f"  데이터 부족으로 건너뜀")
            continue
        
        # 1단계: 학습용 데이터 안전하게 준비
        train_data_safe = make_features(raw_train_data, cutoff_date=cutoff_date)
        print(f"  처리된 학습 데이터: {len(train_data_safe)}행")
        
        # 2단계: 모델 평가 및 선택
        center_results = comprehensive_evaluation_comparison(center_name, train_data_safe)
        all_training_results.extend(center_results)
        
        center_best_models = select_and_train_best_models_safe(center_name, train_data_safe, center_results)
        if center_best_models:
            best_models_by_center[center_name] = center_best_models
        
        # 3단계: 예측용 데이터 준비 (과거 정보만 활용)
        if len(raw_future_data) > 0 and center_name in best_models_by_center:
            future_data_with_features = make_features_for_prediction(raw_train_data, raw_future_data)
            
            # 4단계: 예측 수행
            center_predictions = make_predictions_safe(
                center_name, future_data_with_features, best_models_by_center[center_name]
            )
            all_predictions.extend(center_predictions)
    
    # 5단계: 결과 정리
    if not all_predictions:
        print("예측 결과가 없습니다.")
        return None
    
    final_results_table = create_final_results_table(all_predictions)
    performance_summary = create_performance_summary(final_results_table)
    
    print_final_results(final_results_table, performance_summary)
    save_final_results(final_results_table, performance_summary, pd.DataFrame(all_training_results))
    
    return final_results_table, best_models_by_center, performance_summary

def select_and_train_best_models_safe(center_name, train_data, evaluation_results):
    """안전한 모델 선택 및 학습"""
    
    results_df = pd.DataFrame(evaluation_results)
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) == 0:
        print(f"    성공한 모델이 없습니다.")
        return None
    
    best_models = {}
    
    # 회귀 최고 성능 모델
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"    최고 회귀 모델: {best_reg['model']} (R²={best_reg['r2']:.3f})")
        
        reg_pipeline = retrain_on_full_safe_data(train_data, best_reg['model'], 'regression', best_reg['split_method'])
        
        if reg_pipeline:
            best_models['regression'] = {
                'model_name': best_reg['model'],
                'pipeline': reg_pipeline['pipeline'],
                'feature_names': reg_pipeline['feature_names'],
                'performance': dict(best_reg),
                'split_method': best_reg['split_method']
            }
    
    # 분류 최고 성능 모델
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"    최고 분류 모델: {best_clf['model']} (F1={best_clf['macro_f1']:.3f})")
        
        clf_pipeline = retrain_on_full_safe_data(train_data, best_clf['model'], 'classification', best_clf['split_method'])
        
        if clf_pipeline:
            best_models['classification'] = {
                'model_name': best_clf['model'],
                'pipeline': clf_pipeline['pipeline'],
                'feature_names': clf_pipeline['feature_names'],
                'performance': dict(best_clf),
                'split_method': best_clf['split_method']
            }
    
    return best_models if best_models else None

def retrain_on_full_safe_data(train_data, model_name, model_type, split_method):
    """안전한 전체 데이터 재학습"""
    
    try:
        target_col = "합계_1일후" if model_type == "regression" else "등급_1일후"
        
        # 전체 데이터를 train으로 사용 (test_size를 아주 작게)
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=model_type, 
            test_size=0.01, split_method=split_method
        )
        
        # 전체 결합
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        
        # 모델 학습
        if model_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, model_type)
        pipeline.fit(X_all, y_all)
        
        return {'pipeline': pipeline, 'feature_names': feature_names}
        
    except Exception as e:
        print(f"    모델 재학습 실패 ({model_name}): {e}")
        return None

def make_predictions_safe(center_name, future_data, trained_models):
    """안전한 예측 수행"""
    
    predictions = []
    
    for task_type, model_info in trained_models.items():
        try:
            pipeline = model_info['pipeline']
            feature_names = model_info['feature_names']
            model_name = model_info['model_name']
            
            # 예측 데이터 준비
            X_future = prepare_prediction_features(future_data, feature_names)
            
            if X_future is None or len(X_future) == 0:
                continue
            
            # 예측 수행
            y_pred = pipeline.predict(X_future)
            print(f"    {task_type} 예측 완료: {len(y_pred)}개")
            
            # 결과 저장
            for i in range(len(X_future)):
                pred_result = {
                    'date': future_data.iloc[i]['날짜'],
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_name,
                    'target_column': "합계_1일후" if task_type == "regression" else "등급_1일후",
                    'actual_value': None,  # 운영 환경에서는 모름
                    'predicted_value': float(y_pred[i])
                }
                predictions.append(pred_result)
                
        except Exception as e:
            print(f"    {task_type} 예측 실패: {e}")
    
    return predictions

def prepare_prediction_features(future_data, expected_features):
    """예측용 피처 준비"""
    
    not_use_col = [
        '날짜', '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    # 피처 선택
    available_cols = [col for col in future_data.columns if col not in not_use_col]
    X_future = future_data[available_cols].copy()
    
    # 수치형 변환
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    # 누락된 피처 처리
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    # 피처 순서 맞춤
    X_future = X_future[expected_features].copy()
    
    return X_future

def load_original_data():
    """원본 데이터 로드"""
    nanji_raw = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')
    jungnang_raw = pd.read_csv('../data/processed/center_season/jungnang/중랑_merged.csv', encoding='utf-8-sig')
    seonam_raw = pd.read_csv('../data/processed/center_season/seonam/서남_merged.csv', encoding='utf-8-sig')
    tancheon_raw = pd.read_csv('../data/processed/center_season/tancheon/탄천_merged.csv', encoding='utf-8-sig')
    
    return {
        "nanji": nanji_raw,
        "jungnang": jungnang_raw,
        "seonam": seonam_raw,
        "tancheon": tancheon_raw
    }

# ================================================================================================
# 4. 실행 함수
# ================================================================================================

def run_safe_production_pipeline(cutoff_date='2025-05-20'):
    """안전한 파이프라인 실행"""
    
    print("Data Leakage 방지 파이프라인을 시작합니다...")
    start_time = time.time()
    
    # 파이프라인 실행
    results = complete_production_simulation_safe(cutoff_date=cutoff_date)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\n파이프라인 실행 완료! 소요시간: {elapsed_time:.1f}초")
    
    return results

# ================================================================================================
# 5. 사용 방법
# ================================================================================================

if __name__ == "__main__":
    print("=== Data Leakage 방지 운영 환경 시뮬레이션 ===")
    print()
    print("사용법:")
    print("results = run_safe_production_pipeline(cutoff_date='2025-05-20')")
    print()
    print("특징:")
    print("- 5월 20일까지만 학습용 타겟 변수 생성")
    print("- 5월 21일 이후는 완전히 새로운 데이터로 취급")
    print("- 파생변수는 과거 정보만 활용해서 생성")
    print("- 모델은 절대 재학습하지 않음")
    


In [ ]:
# 기존 모든 코드 위에 이 코드를 추가하고
results = run_safe_production_pipeline(cutoff_date='2025-05-20')

# ================================================================================================
# 최종 파이프라인 실행
# ================================================================================================

# 이 함수 하나만 호출하면 위에서 정의한 모든 과정이 자동으로 실행됩니다.
results = run_safe_production_pipeline(cutoff_date='2025-05-20')

# 결과 확인 (오류 없이 실행 완료되었을 경우)
if results:
    final_table, trained_models, performance_summary = results
    print("\n--- 최종 결과 테이블 미리보기 ---")
    print(final_table.head())